## opXANES，CLAESS-2023, Cell2

In [ ]:
import sys
from pathlib import Path

import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spectrochempy as scp
import xarray as xr
from matplotlib import gridspec, ticker
from matplotlib.colorbar import Colorbar
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from spectrochempy import ur

In [ ]:
# Ensure custom module Path is set before import
sys.path.append(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure")
from colors import tol_cmap, tol_cset  # type: ignore

# 画图的初始设置
plt.style.use(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure\liuchzzyy.mplstyle")
# print(plt.style.available)  # noqa: ERA001

# xarray setting
xr.set_options(
    cmap_sequential="viridis",
    cmap_divergent="viridis",
    display_width=150,
)  # viridis, gray

# 颜色设定
colors = tol_cset("vibrant")
if colors is not None:
    colors = list(colors)
else:
    # Fallback colors in case tol_cset returns None
    colors = ["#0077BB", "#33BBEE", "#009988", "#EE7733", "#CC3311", "#EE3377", "#BBBBBB"]
if r"sunset" not in plt.colormaps():
    cmap = tol_cmap("sunset")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)
if r"rainbow_PuRd" not in plt.colormaps():
    cmap = tol_cmap("rainbow_PuRd")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)  # 备用 plasma

# 输出的文件夹
path_out = Path(r"C:\Users\chengliu\Desktop\Figure")

# Set math font
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "Arial"
mpl.rcParams["mathtext.it"] = "Arial:italic"
mpl.rcParams["mathtext.bf"] = "Arial:bold"
mpl.rcParams["mathtext.sf"] = "Arial"
mpl.rcParams["mathtext.tt"] = "Arial"
mpl.rcParams["mathtext.cal"] = "Arial"
mpl.rcParams["mathtext.default"] = "regular"

### Overview

#### 电化学数据

In [ ]:
# 电化学上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2\Echem"  # noqa: E501, RUF001
)

# 读取电化学数据
with open(path_file.joinpath(r"cell2_03C_0.7mg.txt"), "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break

data = pd.read_csv(
    path_file.joinpath(r"cell2_03C_0.7mg.txt"),
    sep=r"\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    index_col=None,
    decimal=".",
    engine="python",
).dropna(axis=1, how="all")  # noqa: E501
data.columns = [r"cycle number", r"time/s", r"Ewe/V", r"<I>/mA", r"Capacity/mA.h"]
# 转换数据格式
data[["Ewe/V", "<I>/mA", "Capacity/mA.h"]] = data[["Ewe/V", "<I>/mA", "Capacity/mA.h"]].apply(pd.to_numeric, errors="coerce")  # noqa: E501
data["time/s"] = data["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
data["cycle number"] = data["cycle number"].astype(float).astype(np.int16)

data = data[data.iloc[:, 0].isin([0, 1])]

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

labels = [r"First", r"Second", None, None]
for i, idx in enumerate(data.iloc[:, 0].unique()):
    temp = data[data.iloc[:, 0] == idx].reset_index(drop=True)
    # 找到电压最小值的索引
    idx_min = temp["Ewe/V"].idxmin()
    # 断开最小值前后的曲线
    ax.plot(
        temp.loc[: idx_min - 1, "Capacity/mA.h"] * 1000 / 0.744,
        temp.loc[: idx_min - 1, "Ewe/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=labels[i],
        zorder=0,
    )
    ax.plot(
        temp.loc[idx_min + 2 : temp.shape[0] - 5, "Capacity/mA.h"] * 1000 / 0.744,
        temp.loc[idx_min + 2 : temp.shape[0] - 5, "Ewe/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=None,
        zorder=0,
    )

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO_2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 500)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=100, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=50, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=9)
ax.set_ylim(0.80, 2.0)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.0))

ax.tick_params(axis="both", which="both", direction="out", labelsize=9, top=False, right=False)
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 0.6), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.02,
    0.07,
    r"$\mathrm{0.5M \ ZnSO_4 + 0.2M \ MnSO_4, C/5}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_0_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_0_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### 匹配谱线的时间和电化学的时间

In [ ]:
# 读取谱线中时间戳
import re

path_log = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Data"
    ).glob(r"*_log.txt")
)

log_content = []
pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}),\d{3} - INFO     Scan done: .* scans_id: (\d+) - \d+"

for path_file in path_log:
    with open(path_file, "r") as file:
        log_text = file.read()
        # 正则表达式匹配 "Scan done" 行
        matches = re.findall(pattern, log_text)
        log_content.append(matches)

# 将嵌套列表展开为一个扁平列表
flat_data = [item for sublist in log_content for item in sublist]
df = pd.DataFrame(data=flat_data, columns=[r"Time", r"ScanID"])
df["Time"] = pd.to_datetime(df["Time"], format="mixed")
df["ScanID"] = pd.to_numeric(df["ScanID"])

# 每条谱线上的 ScanID 和时间
filelist = list(Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Data\cell2").glob(r"**\*.dat"))
data = []
for file_path in filelist:
    with open(file_path, "r", encoding="utf-8") as f:
        scan_id, element, folder_name = None, None, None
        folder_name = file_path.parts[-3]
        for line in f:
            line = line.strip()  # noqa: PLW2901
            # 提取Extracted scans的数字
            if line.startswith("#S 1 Extracted scans:"):
                match = re.search(r"\[(\d+)\]", line)
                if match:
                    scan_id = match.group(1)
            if line.startswith("#C InputFile:"):
                element = line.split("/")[-1].split(".")[0].split("_")
                if "Mn" in element:
                    element = "Mn"
                elif "Zn" in element:
                    element = "Zn"
                data.append([folder_name, element, scan_id])
                break  # 找到后提前退出循环

df2 = pd.DataFrame(data=data, columns=[r"Folder", r"Element", r"ScanID"])
df2["ScanID"] = pd.to_numeric(df2["ScanID"])

pd.merge(df, df2, on="ScanID", how="inner").sort_values(by=["Time"], ascending=True).to_csv(Path.joinpath(path_out, "Time_Index_Spectrum.csv"), sep=",", header=True, index=False)

In [ ]:
# 电化学上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"
)
echem_file = Path.joinpath(path_file, r"Echem", r"cell2_03C_0.7mg.txt")
with open(echem_file, "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break  # 发现后立即退出循环，提高效率
# 读取电化学数据
echem = pd.read_csv(
    echem_file,
    sep="\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    parse_dates=[1, 2],
    date_format="%m/%d/%y %H:%M:%S.%f",
    index_col=None,
).dropna(axis=1, how="all")
# 转换数据格式
echem["time/s"] = pd.to_datetime(echem["time/s"])
echem[["Ewe/V", "<I>/mA"]] = echem[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
# echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"Time_Index_Spectrum.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
time_spectrum["ScanID"] = pd.to_numeric(time_spectrum["ScanID"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
echem_time = echem["time/s"].values
index_spectrum_Mn = [  # noqa: N816
    np.abs(echem_time - t).argmin() for t in time_spectrum["Time"][time_spectrum["Element"] == r"Mn"].values
]
index_spectrum_Zn = [  # noqa: N816
    np.abs(echem_time - t).argmin() for t in time_spectrum["Time"][time_spectrum["Element"] == r"Zn"].values
]

# 提取对应的电位和电流数据，并按电位排序
index_voltage_Mn = pd.concat(  # noqa: N816
    [
        echem.loc[index_spectrum_Mn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False),
        time_spectrum.loc[:, ["Element", "Folder"]][time_spectrum["Element"] == r"Mn"].reset_index(drop=True),
    ],
    axis=1,
    ignore_index=False,
).sort_values(by="Ewe/V", ascending=True)
index_voltage_Zn = pd.concat(  # noqa: N816
    [
        echem.loc[index_spectrum_Zn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False),
        time_spectrum.loc[:, ["Element", "Folder"]][time_spectrum["Element"] == r"Zn"].reset_index(drop=True),
    ],
    axis=1,
    ignore_index=False,
).sort_values(by="Ewe/V", ascending=True)
index_voltage_Mn.reset_index(drop=False, inplace=True)
index_voltage_Zn.reset_index(drop=False, inplace=True)

In [ ]:
%matplotlib inline

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(echem["time/s"], echem["Ewe/V"], ls="-", lw=1.0, c=colors[0], label=r"Voltage", zorder=0)

# Mn
# for i, j in enumerate([r"cell2_P1_el", r"cell2_P2a_ca", r"cell2_P2b_ca"]):
for i, j in enumerate([r"cell2_P2b_ca",]):
    selected_times = echem["time/s"].loc[index_voltage_Mn["index"][index_voltage_Mn["Folder"] == j]]
    selected_voltages = echem["Ewe/V"].loc[index_voltage_Mn["index"][index_voltage_Mn["Folder"] == j]]
    ax.scatter(selected_times, selected_voltages, c=colors[1 + i], edgecolors="face", alpha=1.0, zorder=1)

# 添加索引文本标注
# 只标记一次，选取 r'cell2_P2b_ca' 的点
Index_special = [0, 11, 24, 36, 50]
for i in [r"cell2_P2b_ca",]:
    Index_special1 = [x * 3 for x in Index_special]
    row_index = np.array([np.abs(index_voltage_Mn[index_voltage_Mn["Folder"] == i].loc[:, "level_0"] - val).argmin() for val in Index_special1])
    row_index = index_voltage_Mn[index_voltage_Mn["Folder"] == r"cell2_P2b_ca"].iloc[row_index, 1]
    for j, idx in enumerate(row_index):
        ax.scatter(echem["time/s"].iloc[idx], echem["Ewe/V"].iloc[idx], c=colors[0], edgecolors="face", marker="o", zorder=2)
        ax.text(
            echem["time/s"].iloc[idx],
            echem["Ewe/V"].iloc[idx] + 0.03,
            str(Index_special[j]),
            fontsize=10,
            verticalalignment="bottom",
            horizontalalignment="right",
        )

# # Zn
# for i, j in enumerate([r"cell2_P1_el", r"cell2_P2a_ca", r"cell2_P2b_ca"]):
#     selected_times = echem["time/s"].loc[index_voltage_Zn["index"][index_voltage_Zn["Folder"] == j]]
#     selected_voltages = echem["Ewe/V"].loc[index_voltage_Zn["index"][index_voltage_Zn["Folder"] == j]]
#     ax.scatter(selected_times, selected_voltages, c=colors[1 + i], edgecolors="face", alpha=0.5, zorder=1, marker="*", s=30)

# # # 添加索引文本标注
# # 只标记一次，选取 r'cell2_P1_el' 的点
# Index_special = [0, 10, 24, 36, 49]
#     Index_special1 = list(map(lambda x: x * 3, Index_special))
#     row_index = np.array([np.abs(index_voltage_Zn[index_voltage_Zn['Folder']==i].loc[:, 'level_0'] - val).argmin() for val in Index_special1])
#     row_index = index_voltage_Zn[index_voltage_Zn['Folder']==r'cell3_P1_el'].iloc[row_index, 1]
#     for j, idx in enumerate(row_index):
#         ax.scatter(echem['time/s'].iloc[idx], echem['Ewe/V'].iloc[idx], c=colors[0], edgecolors='face', marker='o', zorder=2)
#         ax.text(echem['time/s'].iloc[idx], echem['Ewe/V'].iloc[idx]+0.03, str(Index_special[j]), fontsize=10, verticalalignment='bottom', horizontalalignment='right')

ax.set_ylabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_ylim(0.85, 1.9)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# 确保时间刻度从数据最开始时间显示
ax.set_xlim(echem["time/s"].min() - pd.Timedelta(minutes=20), echem["time/s"].max() + pd.Timedelta(minutes=20))
ax.set_xlabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%d %H:%M"))  # 设定日期格式
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 1)))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0, 60, 30)))
plt.xticks(rotation=60, horizontalalignment="right")  # 旋转刻度标签，防止重叠

ax.tick_params(axis="both", which="both", direction="out", labelsize=9, left=True, labelleft=True, top=False, labeltop=False)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1), frameon=False, fontsize=11)

ax.text(
    0.16,
    0.15,
    r"$\text{1.40 mg cm}^{-2}$",
    transform=ax.transAxes,
    fontsize=12,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

ax2 = ax.twinx()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(0.3)

ax2.plot(echem["time/s"], echem["<I>/mA"], ls="--", lw=1.0, c=colors[3], label=r"Current", zorder=0)

ax2.set_ylabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_ylim(-0.2, 0.2)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9, right=True, labelright=True)

ax2.legend(loc="upper left", bbox_to_anchor=(0.65, 1), frameon=False, fontsize=11)

plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### 匹配谱线的时间和电化学的时间 -V0

In [ ]:
# 读取谱线中时间戳
import re

path_log = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Data"
    ).glob(r"*_log.txt")
)

log_content = []
pattern = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}),\d{3} - INFO     Scan done: .* scans_id: (\d+) - \d+"

for path_file in path_log:
    with open(path_file, "r") as file:
        log_text = file.read()
        # 正则表达式匹配 "Scan done" 行
        matches = re.findall(pattern, log_text)
        log_content.append(matches)

# 将嵌套列表展开为一个扁平列表
flat_data = [item for sublist in log_content for item in sublist]
df = pd.DataFrame(data=flat_data, columns=[r"Time", r"ScanID"])
df["Time"] = pd.to_datetime(df["Time"], format="mixed")
df["ScanID"] = pd.to_numeric(df["ScanID"])

# 每条谱线上的 ScanID 和时间
filelist = list(Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Data\cell2").glob(r"**\*.dat"))
data = []
for file_path in filelist:
    with open(file_path, "r", encoding="utf-8") as f:
        scan_id, element, folder_name = None, None, None
        folder_name = file_path.parts[-3]
        for line in f:
            line = line.strip()  # noqa: PLW2901
            # 提取Extracted scans的数字
            if line.startswith("#S 1 Extracted scans:"):
                match = re.search(r"\[(\d+)\]", line)
                if match:
                    scan_id = match.group(1)
            if line.startswith("#C InputFile:"):
                element = line.split("/")[-1].split(".")[0].split("_")
                if "Mn" in element:
                    element = "Mn"
                elif "Zn" in element:
                    element = "Zn"
                data.append([folder_name, element, scan_id])
                break  # 找到后提前退出循环

df2 = pd.DataFrame(data=data, columns=[r"Folder", r"Element", r"ScanID"])
df2["ScanID"] = pd.to_numeric(df2["ScanID"])

pd.merge(df, df2, on="ScanID", how="inner").sort_values(by=["Time"], ascending=True).to_csv(Path.joinpath(path_out, "Time_Index_Spectrum.csv"), sep=",", header=True, index=False)

In [ ]:
# 电化学上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"
)
echem_file = Path.joinpath(path_file, r"Echem", r"cell2_03C_0.7mg.txt")
with open(echem_file, "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break  # 发现后立即退出循环，提高效率
# 读取电化学数据
echem = pd.read_csv(
    echem_file,
    sep="\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    parse_dates=[1, 2],
    date_format="%m/%d/%y %H:%M:%S.%f",
    index_col=None,
).dropna(axis=1, how="all")
# 转换数据格式
echem["time/s"] = pd.to_datetime(echem["time/s"])
echem[["Ewe/V", "<I>/mA"]] = echem[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
# echem.info()

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"Time_Index_Spectrum.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
time_spectrum["ScanID"] = pd.to_numeric(time_spectrum["ScanID"])
# time_spectrum.info()

# 匹配谱线和电化学上的时间
echem_time = echem["time/s"].values
index_spectrum_Mn = [  # noqa: N816
    np.abs(echem_time - t).argmin() for t in time_spectrum["Time"][time_spectrum["Element"] == r"Mn"].values
]
index_spectrum_Zn = [  # noqa: N816
    np.abs(echem_time - t).argmin() for t in time_spectrum["Time"][time_spectrum["Element"] == r"Zn"].values
]

# 提取对应的电位和电流数据，并按电位排序
index_voltage_Mn = pd.concat(  # noqa: N816
    [
        echem.loc[index_spectrum_Mn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False),
        time_spectrum.loc[:, ["Element", "Folder"]][time_spectrum["Element"] == r"Mn"].reset_index(drop=True),
    ],
    axis=1,
    ignore_index=False,
).sort_values(by="Ewe/V", ascending=True)
index_voltage_Zn = pd.concat(  # noqa: N816
    [
        echem.loc[index_spectrum_Zn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False),
        time_spectrum.loc[:, ["Element", "Folder"]][time_spectrum["Element"] == r"Zn"].reset_index(drop=True),
    ],
    axis=1,
    ignore_index=False,
).sort_values(by="Ewe/V", ascending=True)
index_voltage_Mn.reset_index(drop=False, inplace=True)
index_voltage_Zn.reset_index(drop=False, inplace=True)

In [ ]:
%matplotlib inline

# 画图
fig = plt.figure(figsize=(7.0, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.3)

ax.plot(echem["time/s"], echem["Ewe/V"], ls="-", lw=1.0, c=colors[0], label=r"Voltage", zorder=0)

# Mn
# for i, j in enumerate([r"cell2_P1_el", r"cell2_P2a_ca", r"cell2_P2b_ca"]):
for i, j in enumerate([r"cell2_P1_el",]):
    selected_times = echem["time/s"].loc[index_voltage_Mn["index"][index_voltage_Mn["Folder"] == j]]
    selected_voltages = echem["Ewe/V"].loc[index_voltage_Mn["index"][index_voltage_Mn["Folder"] == j]]
    ax.scatter(selected_times, selected_voltages, c=colors[1 + i], edgecolors="face", alpha=1.0, zorder=1)

# 添加索引文本标注
# 只标记一次，选取 r'cell2_P1_el' 的点
Index_special = [0, 10, 24, 36, 50]
for i in [r"cell2_P1_el",]:
    Index_special1 = [x * 3 for x in Index_special]
    row_index = np.array([np.abs(index_voltage_Mn[index_voltage_Mn["Folder"] == i].loc[:, "level_0"] - val).argmin() for val in Index_special1])
    row_index = index_voltage_Mn[index_voltage_Mn["Folder"] == r"cell2_P1_el"].iloc[row_index, 1]
    for j, idx in enumerate(row_index):
        ax.scatter(echem["time/s"].iloc[idx], echem["Ewe/V"].iloc[idx], c=colors[0], edgecolors="face", marker="o", zorder=2)
        ax.text(
            echem["time/s"].iloc[idx],
            echem["Ewe/V"].iloc[idx] + 0.03,
            str(Index_special[j]),
            fontsize=10,
            verticalalignment="bottom",
            horizontalalignment="right",
        )

# # Zn
# for i, j in enumerate([r"cell2_P1_el", r"cell2_P2a_ca", r"cell2_P2b_ca"]):
#     selected_times = echem["time/s"].loc[index_voltage_Zn["index"][index_voltage_Zn["Folder"] == j]]
#     selected_voltages = echem["Ewe/V"].loc[index_voltage_Zn["index"][index_voltage_Zn["Folder"] == j]]
#     ax.scatter(selected_times, selected_voltages, c=colors[1 + i], edgecolors="face", alpha=0.5, zorder=1, marker="*", s=30)

# # # 添加索引文本标注
# # 只标记一次，选取 r'cell2_P1_el' 的点  # noqa: RUF003
# Index_special = [0, 10, 24, 36, 49]  # noqa: ERA001
# for i in [r'cell2_P1_el',]:
#     Index_special1 = list(map(lambda x: x * 3, Index_special))  # noqa: ERA001
#     row_index = np.array([np.abs(index_voltage_Zn[index_voltage_Zn['Folder']==i].loc[:, 'level_0'] - val).argmin() for val in Index_special1])  # noqa: E501, ERA001
#     row_index = index_voltage_Zn[index_voltage_Zn['Folder']==r'cell3_P1_el'].iloc[row_index, 1]  # noqa: ERA001
#     for j, idx in enumerate(row_index):
#         ax.scatter(echem['time/s'].iloc[idx], echem['Ewe/V'].iloc[idx], c=colors[0], edgecolors='face', marker='o', zorder=2)  # noqa: E501, ERA001
#         ax.text(echem['time/s'].iloc[idx], echem['Ewe/V'].iloc[idx]+0.03, str(Index_special[j]), fontsize=10, verticalalignment='bottom', horizontalalignment='right')  # noqa: E501, ERA001

ax.set_ylabel(
    r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$",
    fontsize=11,
)
ax.set_ylim(0.85, 1.9)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

# 确保时间刻度从数据最开始时间显示
ax.set_xlim(echem["time/s"].min() - pd.Timedelta(minutes=20), echem["time/s"].max() + pd.Timedelta(minutes=20))
ax.set_xlabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b-%d %H:%M"))  # 设定日期格式
ax.xaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 1)))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0, 60, 30)))
plt.xticks(rotation=60, horizontalalignment="right")  # 旋转刻度标签，防止重叠

ax.tick_params(axis="both", which="both", direction="out", labelsize=9, left=True, labelleft=True, top=False, labeltop=False)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1), frameon=False, fontsize=11)

ax.text(
    0.16,
    0.15,
    r"$\text{1.40 mg cm}^{-2}$",
    transform=ax.transAxes,
    fontsize=12,
    color=colors[3],
    va="top",
    ha="right",
    fontfamily="Arial",
)

ax2 = ax.twinx()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(0.3)

ax2.plot(echem["time/s"], echem["<I>/mA"], ls="--", lw=1.0, c=colors[3], label=r"Current", zorder=0)

ax2.set_ylabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_ylim(-0.2, 0.2)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(axis="both", which="both", direction="out", labelsize=9, right=True, labelright=True)

ax2.legend(loc="upper left", bbox_to_anchor=(0.65, 1), frameon=False, fontsize=11)

plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### P1

In [ ]:
# 读取 reference + operando 数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"  # noqa: RUF001
)

data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Mn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Mn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Mn = pd.concat([data1, data2.iloc[:, 5:]], axis=1, ignore_index=True)  # noqa: N816
pdf_Mn = xas_Mn.iloc[:, 0:5]  # noqa: N816
pdf_Mn.columns = [
    r"Energy",
    r"0.2M_MnSO4(aq.)",
    r"alpha_MnO2_Electrode",
    r"alpha_MnO2_Powder",
    r"FC1st",
]
opData_Mn = xas_Mn.iloc[:, [0, *list(range(5, xas_Mn.shape[1]))]]  # noqa: N816
opData_Mn.columns = list(range(0, opData_Mn.shape[1], 1))

data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Zn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Zn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Zn = pd.concat([data1, data2.iloc[:, 3:]], axis=1, ignore_index=True)  # noqa: N816
pdf_Zn = xas_Zn.iloc[:, 0:3]  # noqa: N816
pdf_Zn.columns = [r"Energy", r"0.5M_ZnSO4(aq.)", r"ZHS"]
opData_Zn = xas_Zn.iloc[:, [0, *list(range(3, xas_Zn.shape[1]))]]  # noqa: N816
opData_Zn.columns = list(range(0, opData_Zn.shape[1], 1))

# EXAFS
data1 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Mn_Oct2022_1.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Mn_Oct2022_2.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
EXAFS_Mn = pd.concat([data1, data2.iloc[:, 5:]], axis=1, ignore_index=True)  # noqa: N816
pdf2_Mn = EXAFS_Mn.iloc[:, 0:5]  # noqa: N816
pdf2_Mn.columns = [
    r"Distance",
    r"0.2M_MnSO4(aq.)",
    r"alpha_MnO2_Electrode",
    r"alpha_MnO2_Powder",
    r"FC1st",
]
opData2_Mn = EXAFS_Mn.iloc[:, [0, *list(range(5, EXAFS_Mn.shape[1]))]]  # noqa: N816
opData2_Mn.columns = list(range(0, opData2_Mn.shape[1], 1))

data1 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Zn_Oct2022_1.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Zn_Oct2022_2.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
EXAFS_Zn = pd.concat([data1, data2.iloc[:, 3:]], axis=1, ignore_index=True)  # noqa: N816
pdf2_Zn = EXAFS_Zn.iloc[:, 0:3]  # noqa: N816
pdf2_Zn.columns = [r"Distance", r"0.5M_ZnSO4(aq.)", r"ZHS"]
opData2_Zn = EXAFS_Zn.iloc[:, [0, *list(range(3, EXAFS_Zn.shape[1]))]]  # noqa: N816
opData2_Zn.columns = list(range(0, opData2_Zn.shape[1], 1))

In [ ]:
# 电化学上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"  # noqa: RUF001
)
echem_file = Path.joinpath(path_file, r"Echem", r"cell2_03C_0.7mg.txt")
with open(echem_file, "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break  # 发现后立即退出循环，提高效率
# 读取电化学数据
echem = pd.read_csv(
    echem_file,
    sep="\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    parse_dates=[1, 2],
    date_format="%m/%d/%y %H:%M:%S.%f",
    index_col=None,
).dropna(axis=1, how="all")
# 转换数据格式
echem["time/s"] = pd.to_datetime(echem["time/s"])
echem[["Ewe/V", "<I>/mA"]] = echem[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
# echem.info()  # noqa: ERA001

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"Time_Index_Spectrum.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
time_spectrum["ScanID"] = pd.to_numeric(time_spectrum["ScanID"])
# time_spectrum.info()  # noqa: ERA001

In [ ]:
# 提取出 P1 的电位和电流数据
time_spectrum = time_spectrum[time_spectrum["Folder"] == r"cell2_P1_el"]  # cell2_P1_el , cell2_P2a_ca , cell2_P2b_ca

# 匹配谱线和电化学上的时间
echem_time = echem["time/s"].values
index_spectrum_Mn = [  # noqa: N816
    np.abs(echem_time - t).argmin() for t in time_spectrum["Time"][time_spectrum["Element"] == r"Mn"].values
]
index_spectrum_Zn = [  # noqa: N816
    np.abs(echem_time - t).argmin() for t in time_spectrum["Time"][time_spectrum["Element"] == r"Zn"].values
]

# 匹配谱线只有一条 OCV 数据， 但是时间上是有两条的时间
index_spectrum_Mn = index_spectrum_Mn[2:]  # noqa: N816
index_spectrum_Zn = index_spectrum_Zn[2:]  # noqa: N816

# 提取对应的电位和电流数据，并按电位排序
index_voltage_Mn = pd.concat(  # noqa: N816
    [
        echem.loc[index_spectrum_Mn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False),
        time_spectrum.loc[:, ["Element", "Folder"]][time_spectrum["Element"] == r"Mn"].reset_index(drop=True),
    ],
    axis=1,
    ignore_index=False,
).sort_values(by="Ewe/V", ascending=True)
index_voltage_Zn = pd.concat(  # noqa: N816
    [
        echem.loc[index_spectrum_Zn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False),
        time_spectrum.loc[:, ["Element", "Folder"]][time_spectrum["Element"] == r"Zn"].reset_index(drop=True),
    ],
    axis=1,
    ignore_index=False,
).sort_values(by="Ewe/V", ascending=True)
index_voltage_Mn.reset_index(drop=False, inplace=True)
index_voltage_Zn.reset_index(drop=False, inplace=True)

# 选择需要的电化学数据以及对应的谱线
# Mn
index = [0, 9, 22, 35, 48]
selected_Mn = (  # noqa: N816
    index_voltage_Mn[index_voltage_Mn["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_Mn["index"] = pd.to_numeric(selected_Mn["index"], downcast="integer")
selected_echem_Mn = echem[echem.index <= selected_Mn["index"].iloc[-1]]  # noqa: N816
mapping_spectum_Mn = opData_Mn.loc[:, opData_Mn.columns <= selected_Mn["level_0"].iloc[-1] + 1]  # noqa: N816
mapping_spectum_Mn2 = opData2_Mn.loc[:, opData2_Mn.columns <= selected_Mn["level_0"].iloc[-1] + 1]  # noqa: N816

# Zn
index = [0, 9, 22, 35, 48]
selected_Zn = (  # noqa: N816
    index_voltage_Zn[index_voltage_Zn["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_Zn["index"] = pd.to_numeric(selected_Zn["index"], downcast="integer")
selected_echem_Zn = echem[echem.index <= selected_Zn["index"].iloc[-1]]  # noqa: N816
mapping_spectum_Zn = opData_Zn.loc[:, opData_Zn.columns <= selected_Zn["level_0"].iloc[-1] + 1]  # noqa: N816
mapping_spectum_Zn2 = opData2_Zn.loc[:, opData2_Zn.columns <= selected_Zn["level_0"].iloc[-1] + 1]  # noqa: N816

# 提取特定位置的值
# Mn
Intensityslice_Mn = [mapping_spectum_Mn.iloc[np.abs(mapping_spectum_Mn.iloc[:, 0] - t).argmin()] for t in [6553.09, 6561.67]]
Intensityslice_Zn = [
    mapping_spectum_Zn.iloc[np.abs(mapping_spectum_Zn.iloc[:, 0] - t).argmin()]
    for t in [
        9668.82,
    ]
]
# Zn
Intensityslice_Mn2 = [mapping_spectum_Mn2.iloc[np.abs(mapping_spectum_Mn2.iloc[:, 0] - t).argmin()] for t in [2.40, 3.01]]
Intensityslice_Zn2 = [
    mapping_spectum_Zn2.iloc[np.abs(mapping_spectum_Zn2.iloc[:, 0] - t).argmin()]
    for t in [
        1.59,
    ]
]

In [ ]:
# Zn satuation
index_satuation = np.arange(17, 23, 1)
mapping_spectum_Zn2.drop(index=index_satuation, axis=1, inplace=True)  # type: ignore
Intensityslice_Zn2[0].drop(index=index_satuation, axis=0, inplace=True)  # type: ignore

In [ ]:
# 绘图
%matplotlib inline
plt.close("all")
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 3, width_ratios=[1, 3, 3], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

# Mn
specical_index_Mn = [0, 9, 22, 35, 48]  # noqa: N816
ax.plot(selected_echem_Mn["Ewe/V"], selected_echem_Mn["time/s"], ls="-", lw=1.0, c=colors[0], label=r"opCoinB", zorder=1)
ax.scatter(
    selected_echem_Mn["Ewe/V"].iloc[selected_Mn["index"]],
    selected_echem_Mn["time/s"].iloc[selected_Mn["index"]],
    edgecolors="face",
    c=colors[1],
    zorder=1,
)

# 特殊位置颜色标定
for idx in specical_index_Mn:
    temp = selected_Mn["index"][selected_Mn["level_0"] == idx]
    ax.scatter(
        selected_echem_Mn["Ewe/V"].iloc[temp],
        selected_echem_Mn["time/s"].iloc[temp],
        c=colors[0],
        edgecolors="face",
        marker="o",
        zorder=2,
    )

# 添加索引文本标注
for i in range(selected_Mn.shape[0]):
    temp = selected_Mn["index"].iloc[i]
    ax.text(
        selected_echem_Mn["Ewe/V"].iloc[temp] + 0.15,
        selected_echem_Mn["time/s"].iloc[temp],
        str(selected_Mn["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
        zorder=1,
    )

# Zn
specical_index_Zn = [0, 9, 22, 35, 48]  # noqa: N816
ax.plot(selected_echem_Zn["Ewe/V"], selected_echem_Zn["time/s"], ls="-", lw=1.0, c=colors[3], label=r"opCoinB", zorder=0)
ax.scatter(
    selected_echem_Zn["Ewe/V"].iloc[selected_Zn["index"]],
    selected_echem_Zn["time/s"].iloc[selected_Zn["index"]],
    edgecolors="face",
    c=colors[2],
    zorder=1,
)

# # 特殊位置颜色标定
# for idx in specical_index_Zn:
#     temp = selected_Zn['index'][selected_Zn['level_0'] == idx]  # noqa: ERA001
#     ax.scatter(selected_echem_Zn['Ewe/V'].iloc[temp], selected_echem_Zn['time/s'].iloc[temp], c=colors[2], edgecolors='face', marker='o')  # noqa: E501, ERA001

# # 添加索引文本标注
# for i in range(selected_Zn.shape[0]):
#     temp = selected_Zn['index'].iloc[i]  # noqa: ERA001
#     ax.text(selected_echem_Zn['Ewe/V'].iloc[temp] + 0.15, selected_echem_Zn['time/s'].iloc[temp],
#             str(selected_Zn['level_0'].iloc[i]), fontsize=10, verticalalignment='bottom', horizontalalignment='right', zorder=2)  # noqa: E501

ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11)
ax.set_xlim(0.8, 2.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))

# 确保时间刻度从数据最开始时间显示
ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(
    selected_echem_Mn["time/s"].min() - pd.Timedelta(minutes=20),
    selected_echem_Mn["time/s"].max() + pd.Timedelta(minutes=20),
)
ax.yaxis.set_major_formatter(mdates.DateFormatter("%b-%d %H:%M"))  # 设定日期格式
ax.yaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 1)))
ax.yaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0, 60, 30)))
plt.yticks(rotation=0, horizontalalignment="right")
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    left=True,
    labelleft=True,
    right=False,
    labelright=False,
    top=False,
    labeltop=False,
)

ax2 = ax.twiny()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(2.4)
ax2.plot(selected_echem_Mn["<I>/mA"], selected_echem_Mn["time/s"], ls="--", lw=1.0, c=colors[3], label=r"Current")
ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.1, 0.1)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=True,
    labeltop=True,
    right=False,
    labelright=False,
    left=False,
    labelleft=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.2, 0, 1, 1))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf_Mn.iloc[:, 0], pdf_Mn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[0][i], label=labels[0][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Mn["level_0"].isin(specical_index_Mn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Mn.shape[0]):
    temp = mapping_spectum_Mn.iloc[:, mapping_spectum_Mn.columns == (selected_Mn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Mn.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Mn["level_0"].iloc[i]))

ax.set_xlim(6530, 6630)
ax.set_xlabel("Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=-10))

ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 图 C
subfig = fig.add_subfigure(gs[0, 2])
ax = subfig.add_axes((-0.55, 0, 1, 1))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf_Zn.iloc[:, 0], pdf_Zn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[1][i], label=labels[1][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Zn["level_0"].isin(specical_index_Zn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Zn.shape[0]):
    temp = mapping_spectum_Zn.iloc[:, mapping_spectum_Zn.columns == (selected_Zn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Zn.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Zn["level_0"].iloc[i]))

ax.set_xlim(9640, 9740)
ax.set_xlabel("Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=0))

ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 图 D
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.45, 1.4, 1.4))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf2_Mn.iloc[:, 0], pdf2_Mn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[0][i], label=labels[0][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Mn["level_0"].isin(specical_index_Mn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Mn.shape[0]):
    temp = mapping_spectum_Mn2.iloc[:, mapping_spectum_Mn2.columns == (selected_Mn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Mn2.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Mn["level_0"].iloc[i]))

ax.set_xlim(0, 6)
ax.set_xlabel(r"R ($\mathrm{\AA}$)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=1, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.5, offset=-10))

ax.set_ylim(0, 2.4)
ax.set_ylabel(r"FT [$\mathrm{{\chi}({\kappa})*{\kappa}{^2}}$]", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 图 E
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.05, -0.2, 0.9, 0.9))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf2_Zn.iloc[:, 0], pdf2_Zn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[1][i], label=labels[1][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Zn["level_0"].isin(specical_index_Zn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Zn.shape[0]):
    temp = mapping_spectum_Zn2.iloc[:, mapping_spectum_Zn2.columns == (selected_Zn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Zn2.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Zn["level_0"].iloc[i]))

ax.set_xlim(0, 6)
ax.set_xlabel(r"R ($\mathrm{\AA}$)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=1, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.5, offset=-10))

ax.set_ylim(0, 1.5)
ax.set_ylabel(r"FT [$\mathrm{{\chi}({\kappa})*{\kappa}{^2}}$]", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 保存图像
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P1_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P1_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### P2a

In [ ]:
# 读取 reference + operando 数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"  # noqa: RUF001
)

data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Mn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Mn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Mn = pd.concat([data1, data2.iloc[:, 5:]], axis=1, ignore_index=True)  # noqa: N816
pdf_Mn = xas_Mn.iloc[:, 0:5]  # noqa: N816
pdf_Mn.columns = [
    r"Energy",
    r"0.2M_MnSO4(aq.)",
    r"alpha_MnO2_Electrode",
    r"alpha_MnO2_Powder",
    r"FC1st",
]
opData_Mn = xas_Mn.iloc[:, [0, *list(range(5, xas_Mn.shape[1]))]]  # noqa: N816
opData_Mn.columns = list(range(0, opData_Mn.shape[1], 1))

data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Zn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Zn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Zn = pd.concat([data1, data2.iloc[:, 3:]], axis=1, ignore_index=True)  # noqa: N816
pdf_Zn = xas_Zn.iloc[:, 0:3]  # noqa: N816
pdf_Zn.columns = [r"Energy", r"0.5M_ZnSO4(aq.)", r"ZHS"]
opData_Zn = xas_Zn.iloc[:, [0, *list(range(3, xas_Zn.shape[1]))]]  # noqa: N816
opData_Zn.columns = list(range(0, opData_Zn.shape[1], 1))

# EXAFS
data1 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Mn_Oct2022_1.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Mn_Oct2022_2.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
EXAFS_Mn = pd.concat([data1, data2.iloc[:, 5:]], axis=1, ignore_index=True)  # noqa: N816
pdf2_Mn = EXAFS_Mn.iloc[:, 0:5]  # noqa: N816
pdf2_Mn.columns = [
    r"Distance",
    r"0.2M_MnSO4(aq.)",
    r"alpha_MnO2_Electrode",
    r"alpha_MnO2_Powder",
    r"FC1st",
]
opData2_Mn = EXAFS_Mn.iloc[:, [0, *list(range(5, EXAFS_Mn.shape[1]))]]  # noqa: N816
opData2_Mn.columns = list(range(0, opData2_Mn.shape[1], 1))

data1 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Zn_Oct2022_1.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Zn_Oct2022_2.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
EXAFS_Zn = pd.concat([data1, data2.iloc[:, 3:]], axis=1, ignore_index=True)  # noqa: N816
pdf2_Zn = EXAFS_Zn.iloc[:, 0:3]  # noqa: N816
pdf2_Zn.columns = [r"Distance", r"0.5M_ZnSO4(aq.)", r"ZHS"]
opData2_Zn = EXAFS_Zn.iloc[:, [0, *list(range(3, EXAFS_Zn.shape[1]))]]  # noqa: N816
opData2_Zn.columns = list(range(0, opData2_Zn.shape[1], 1))

In [ ]:
# 电化学上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"  # noqa: RUF001
)
echem_file = Path.joinpath(path_file, r"Echem", r"cell2_03C_0.7mg.txt")
with open(echem_file, "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break  # 发现后立即退出循环，提高效率
# 读取电化学数据
echem = pd.read_csv(
    echem_file,
    sep=r"\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    parse_dates=[1, 2],
    date_format="%m/%d/%y %H:%M:%S.%f",
    index_col=None,
    engine="python",
).dropna(axis=1, how="all")
# 转换数据格式
echem["time/s"] = pd.to_datetime(echem["time/s"])
echem[["Ewe/V", "<I>/mA"]] = echem[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
# echem.info()  # noqa: ERA001

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"Time_Index_Spectrum.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
time_spectrum["ScanID"] = pd.to_numeric(time_spectrum["ScanID"])
# time_spectrum.info()  # noqa: ERA001


In [ ]:
# 提取出 P2a 的电位和电流数据
time_spectrum = time_spectrum[time_spectrum["Folder"] == r"cell2_P2a_ca"]  # cell2_P1_el , cell2_P2a_ca , cell2_P2b_ca

# 匹配谱线和电化学上的时间
echem_time = echem["time/s"].values
index_spectrum_Mn = [  # noqa: N816
    np.abs(echem_time - t).argmin() for t in time_spectrum["Time"][time_spectrum["Element"] == r"Mn"].values
]
index_spectrum_Zn = [  # noqa: N816
    np.abs(echem_time - t).argmin() for t in time_spectrum["Time"][time_spectrum["Element"] == r"Zn"].values
]

# 匹配谱线只有一条 OCV 数据， 但是时间上是有两条的时间
index_spectrum_Mn = index_spectrum_Mn[1:]  # noqa: N816
index_spectrum_Zn = index_spectrum_Zn[1:]  # noqa: N816

# 提取对应的电位和电流数据，并按电位排序
index_voltage_Mn = pd.concat(  # noqa: N816
    [
        echem.loc[index_spectrum_Mn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False),
        time_spectrum.loc[:, ["Element", "Folder"]][time_spectrum["Element"] == r"Mn"].reset_index(drop=True),
    ],
    axis=1,
    ignore_index=False,
).sort_values(by="Ewe/V", ascending=True)
index_voltage_Zn = pd.concat(  # noqa: N816
    [
        echem.loc[index_spectrum_Zn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False),
        time_spectrum.loc[:, ["Element", "Folder"]][time_spectrum["Element"] == r"Zn"].reset_index(drop=True),
    ],
    axis=1,
    ignore_index=False,
).sort_values(by="Ewe/V", ascending=True)
index_voltage_Mn.reset_index(drop=False, inplace=True)
index_voltage_Zn.reset_index(drop=False, inplace=True)

# 选择需要的电化学数据以及对应的谱线
# Mn
index = [0, 9, 22, 35, 48]
selected_Mn = (  # noqa: N816
    index_voltage_Mn[index_voltage_Mn["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_Mn["index"] = pd.to_numeric(selected_Mn["index"], downcast="integer")
selected_echem_Mn = echem[echem.index <= selected_Mn["index"].iloc[-1]]  # noqa: N816
mapping_spectum_Mn = opData_Mn.loc[:, opData_Mn.columns <= selected_Mn["level_0"].iloc[-1] + 1]  # noqa: N816
mapping_spectum_Mn2 = opData2_Mn.loc[:, opData2_Mn.columns <= selected_Mn["level_0"].iloc[-1] + 1]  # noqa: N816

# Zn
index = [0, 9, 22, 35, 48]
selected_Zn = (  # noqa: N816
    index_voltage_Zn[index_voltage_Zn["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_Zn["index"] = pd.to_numeric(selected_Zn["index"], downcast="integer")
selected_echem_Zn = echem[echem.index <= selected_Zn["index"].iloc[-1]]  # noqa: N816
mapping_spectum_Zn = opData_Zn.loc[:, opData_Zn.columns <= selected_Zn["level_0"].iloc[-1] + 1]  # noqa: N816
mapping_spectum_Zn2 = opData2_Zn.loc[:, opData2_Zn.columns <= selected_Zn["level_0"].iloc[-1] + 1]  # noqa: N816

# 提取特定位置的值
# Mn
Intensityslice_Mn = [mapping_spectum_Mn.iloc[np.abs(mapping_spectum_Mn.iloc[:, 0] - t).argmin()] for t in [6553.09, 6561.67]]
Intensityslice_Zn = [mapping_spectum_Zn.iloc[np.abs(mapping_spectum_Zn.iloc[:, 0] - t).argmin()] for t in [9668.82]]
# Zn
Intensityslice_Mn2 = [mapping_spectum_Mn2.iloc[np.abs(mapping_spectum_Mn2.iloc[:, 0] - t).argmin()] for t in [2.40, 3.01]]
Intensityslice_Zn2 = [mapping_spectum_Zn2.iloc[np.abs(mapping_spectum_Zn2.iloc[:, 0] - t).argmin()] for t in [1.59]]

In [ ]:
# Zn satuation
index_satuation = np.arange(17, 23, 1)
mapping_spectum_Zn2.drop(index_satuation, axis=1, inplace=True)  # type: ignore
Intensityslice_Zn2[0].drop(index=index_satuation, axis=0, inplace=True)  # type: ignore

In [ ]:
# 绘图
%matplotlib inline
plt.close("all")
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 3, width_ratios=[1, 3, 3], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

# Mn
specical_index_Mn = [0, 9, 22, 35, 48]  # noqa: N816
ax.plot(selected_echem_Mn["Ewe/V"], selected_echem_Mn["time/s"], ls="-", lw=1.0, c=colors[0], label=r"Cell3", zorder=1)
ax.scatter(
    selected_echem_Mn["Ewe/V"].iloc[selected_Mn["index"]],
    selected_echem_Mn["time/s"].iloc[selected_Mn["index"]],
    edgecolors="face",
    c=colors[1],
    zorder=1,
)

# 特殊位置颜色标定
for idx in specical_index_Mn:
    temp = selected_Mn["index"][selected_Mn["level_0"] == idx]
    ax.scatter(
        selected_echem_Mn["Ewe/V"].iloc[temp],
        selected_echem_Mn["time/s"].iloc[temp],
        c=colors[0],
        edgecolors="face",
        marker="o",
        zorder=2,
    )

# 添加索引文本标注
for i in range(selected_Mn.shape[0]):
    temp = selected_Mn["index"].iloc[i]
    ax.text(
        selected_echem_Mn["Ewe/V"].iloc[temp] + 0.15,
        selected_echem_Mn["time/s"].iloc[temp],
        str(selected_Mn["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
        zorder=1,
    )

# Zn
specical_index_Zn = [0, 9, 22, 35, 48]  # noqa: N816
ax.plot(selected_echem_Zn["Ewe/V"], selected_echem_Zn["time/s"], ls="-", lw=1.0, c=colors[3], label=r"Cell3", zorder=0)
ax.scatter(
    selected_echem_Zn["Ewe/V"].iloc[selected_Zn["index"]],
    selected_echem_Zn["time/s"].iloc[selected_Zn["index"]],
    edgecolors="face",
    c=colors[2],
    zorder=1,
)

# # 特殊位置颜色标定
# for idx in specical_index_Zn:
#     temp = selected_Zn['index'][selected_Zn['level_0'] == idx]  # noqa: ERA001
#     ax.scatter(selected_echem_Zn['Ewe/V'].iloc[temp], selected_echem_Zn['time/s'].iloc[temp], c=colors[2], edgecolors='face', marker='o')  # noqa: E501, ERA001

# # 添加索引文本标注
# for i in range(selected_Zn.shape[0]):
#     temp = selected_Zn['index'].iloc[i]  # noqa: ERA001
#     ax.text(selected_echem_Zn['Ewe/V'].iloc[temp] + 0.15, selected_echem_Zn['time/s'].iloc[temp],
#             str(selected_Zn['level_0'].iloc[i]), fontsize=10, verticalalignment='bottom', horizontalalignment='right', zorder=2)  # noqa: E501

ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11)
ax.set_xlim(0.8, 2.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))

# 确保时间刻度从数据最开始时间显示
ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(
    selected_echem_Mn["time/s"].min() - pd.Timedelta(minutes=20),
    selected_echem_Mn["time/s"].max() + pd.Timedelta(minutes=20),
)
ax.yaxis.set_major_formatter(mdates.DateFormatter("%b-%d %H:%M"))  # 设定日期格式
ax.yaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 1)))
ax.yaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0, 60, 30)))
plt.yticks(rotation=0, horizontalalignment="right")
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    left=True,
    labelleft=True,
    right=False,
    labelright=False,
    top=False,
    labeltop=False,
)

ax2 = ax.twiny()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(2.4)
ax2.plot(selected_echem_Mn["<I>/mA"], selected_echem_Mn["time/s"], ls="--", lw=1.0, c=colors[3], label=r"Current")  # type: ignore
ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.1, 0.1)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=True,
    labeltop=True,
    right=False,
    labelright=False,
    left=False,
    labelleft=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.15, 0, 1, 1))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf_Mn.iloc[:, 0], pdf_Mn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[0][i], label=labels[0][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3, handletextpad=0.2)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Mn["level_0"].isin(specical_index_Mn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Mn.shape[0]):
    temp = mapping_spectum_Mn.iloc[:, mapping_spectum_Mn.columns == (selected_Mn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Mn.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Mn["level_0"].iloc[i]))

ax.set_xlim(6530, 6630)
ax.set_xlabel("Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=-10))

ax.set_ylim(0, 2.0)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 图 C
subfig = fig.add_subfigure(gs[0, 2])
ax = subfig.add_axes((-0.49, 0, 1, 1))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf_Zn.iloc[:, 0], pdf_Zn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[1][i], label=labels[1][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3, handletextpad=0.2)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Zn["level_0"].isin(specical_index_Zn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Zn.shape[0]):
    temp = mapping_spectum_Zn.iloc[:, mapping_spectum_Zn.columns == (selected_Zn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Zn.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Zn["level_0"].iloc[i]))

ax.set_xlim(9640, 9740)
ax.set_xlabel("Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=0))

ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 图 D
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.45, 1.4, 1.4))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf2_Mn.iloc[:, 0], pdf2_Mn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[0][i], label=labels[0][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3, handletextpad=0.2)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Mn["level_0"].isin(specical_index_Mn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Mn.shape[0]):
    temp = mapping_spectum_Mn2.iloc[:, mapping_spectum_Mn2.columns == (selected_Mn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Mn2.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Mn["level_0"].iloc[i]))

ax.set_xlim(0, 6)
ax.set_xlabel(r"R ($\mathrm{\AA}$)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=1, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.5, offset=-10))

ax.set_ylim(0, 2.4)
ax.set_ylabel(r"FT [$\mathrm{{\chi}({\kappa})*{\kappa}{^2}}$]", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 图 E
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.03, -0.2, 0.9, 0.9))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf2_Zn.iloc[:, 0], pdf2_Zn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[1][i], label=labels[1][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3, handletextpad=0.2)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Zn["level_0"].isin(specical_index_Zn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Zn.shape[0]):
    temp = mapping_spectum_Zn2.iloc[:, mapping_spectum_Zn2.columns == (selected_Zn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Zn2.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Zn["level_0"].iloc[i]))

ax.set_xlim(0, 6)
ax.set_xlabel(r"R ($\mathrm{\AA}$)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=1, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.5, offset=-10))

ax.set_ylim(0, 1.5)
ax.set_ylabel(r"FT [$\mathrm{{\chi}({\kappa})*{\kappa}{^2}}$]", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 图 F
subfig = fig.add_subfigure(gs[1, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.4, -0.2, 0.9, 0.9))
ax.set_box_aspect(0.8)

for i in range(len(Intensityslice_Mn)):
    ax.scatter(
        np.arange(len(Intensityslice_Mn[i]) - 1),
        Intensityslice_Mn[i][1:],
        s=30,
        ls="-",
        edgecolors="face",
        lw=1,
        c=colors[0],
        label=None,
        alpha=1 - 0.3 * i,
    )
    ax.plot(
        np.arange(len(Intensityslice_Mn[i]) - 1),
        Intensityslice_Mn[i][1:],
        ls="-",
        lw=1.0,
        c=colors[0],
        label=None,
        alpha=1 - 0.3 * i,
    )

for i in range(len(Intensityslice_Zn)):
    ax.scatter(
        np.arange(len(Intensityslice_Zn[i]) - 1),
        Intensityslice_Zn[i][1:],
        s=30,
        edgecolors="face",
        ls="-",
        lw=1.0,
        c=colors[i + 2],
        label=None,
    )
    ax.plot(
        np.arange(len(Intensityslice_Zn[i]) - 1),
        Intensityslice_Zn[i][1:],
        ls="-",
        lw=1.0,
        c=colors[i + 2],
        label=None,
        alpha=1 - 0.3 * i,
    )

ax.set_xlim(0, 50)
ax.set_xlabel("Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))

ax.set_ylim(0.8, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

ax2 = ax.twinx()
ax2.set_position((-0.4, -0.2, 0.9, 0.9))
ax2.set_box_aspect(0.8)
for i in range(len(Intensityslice_Mn2)):
    ax2.scatter(  # type: ignore
        np.arange(len(Intensityslice_Mn2[i]) - 1),
        Intensityslice_Mn2[i][1:],
        ls="-",
        lw=1.0,
        c=colors[3],
        marker="*",
        label=None,
        s=100,
        edgecolors="face",
        alpha=1 - 0.3 * i,
    )
    ax2.plot(  # type: ignore
        np.arange(len(Intensityslice_Mn2[i]) - 1),
        Intensityslice_Mn2[i][1:],
        ls="-",
        lw=1.0,
        c=colors[3],
        label=None,
        alpha=1 - 0.3 * i,
    )
for i in range(len(Intensityslice_Zn2)):
    ax2.scatter(  # type: ignore
        np.arange(len(Intensityslice_Zn2[i]) - 1),
        Intensityslice_Zn2[i][1:],
        ls="-",
        lw=1.0,
        marker="*",
        c=colors[4 + i],
        label=None,
        s=100,
        edgecolors="face",
    )
    ax2.plot(  # type: ignore
        np.arange(len(Intensityslice_Zn2[i]) - 1),
        Intensityslice_Zn2[i][1:],
        ls="-",
        lw=1.0,
        c=colors[4 + i],
        label=None,
    )

ax2.set_ylim(0, 1.5)
ax2.set_ylabel(r"FT [$\mathrm{{\chi}({\kappa})*{\kappa}{^2}}$]", fontsize=11)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=True,
    labelright=True,
    bottom=True,
    labelbottom=True,
    left=False,
    labelleft=False,
)


# 保存图像
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P2a_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P2a_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### P2b

In [ ]:
# 读取 reference + operando 数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"  # noqa: RUF001
)

data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Mn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Mn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Mn = pd.concat([data1, data2.iloc[:, 5:]], axis=1, ignore_index=True)  # noqa: N816
pdf_Mn = xas_Mn.iloc[:, 0:5]  # noqa: N816
pdf_Mn.columns = [
    r"Energy",
    r"0.2M_MnSO4(aq.)",
    r"alpha_MnO2_Electrode",
    r"alpha_MnO2_Powder",
    r"FC1st",
]
opData_Mn = xas_Mn.iloc[:, [0, *list(range(5, xas_Mn.shape[1]))]]  # noqa: N816
opData_Mn.columns = list(range(0, opData_Mn.shape[1], 1))

data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Zn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Zn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Zn = pd.concat([data1, data2.iloc[:, 3:]], axis=1, ignore_index=True)  # noqa: N816
pdf_Zn = xas_Zn.iloc[:, 0:3]  # noqa: N816
pdf_Zn.columns = [r"Energy", r"0.5M_ZnSO4(aq.)", r"ZHS"]
opData_Zn = xas_Zn.iloc[:, [0, *list(range(3, xas_Zn.shape[1]))]]  # noqa: N816
opData_Zn.columns = list(range(0, opData_Zn.shape[1], 1))

# EXAFS
data1 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Mn_Oct2022_1.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Mn_Oct2022_2.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
EXAFS_Mn = pd.concat([data1, data2.iloc[:, 5:]], axis=1, ignore_index=True)  # noqa: N816
pdf2_Mn = EXAFS_Mn.iloc[:, 0:5]  # noqa: N816
pdf2_Mn.columns = [
    r"Distance",
    r"0.2M_MnSO4(aq.)",
    r"alpha_MnO2_Electrode",
    r"alpha_MnO2_Powder",
    r"FC1st",
]
opData2_Mn = EXAFS_Mn.iloc[:, [0, *list(range(5, EXAFS_Mn.shape[1]))]]  # noqa: N816
opData2_Mn.columns = list(range(0, opData2_Mn.shape[1], 1))

data1 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Zn_Oct2022_1.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Zn_Oct2022_2.chir2_mag"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
EXAFS_Zn = pd.concat([data1, data2.iloc[:, 3:]], axis=1, ignore_index=True)  # noqa: N816
pdf2_Zn = EXAFS_Zn.iloc[:, 0:3]  # noqa: N816
pdf2_Zn.columns = [r"Distance", r"0.5M_ZnSO4(aq.)", r"ZHS"]
opData2_Zn = EXAFS_Zn.iloc[:, [0, *list(range(3, EXAFS_Zn.shape[1]))]]  # noqa: N816
opData2_Zn.columns = list(range(0, opData2_Zn.shape[1], 1))

In [ ]:
# 电化学上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"  # noqa: RUF001
)
echem_file = Path.joinpath(path_file, r"Echem", r"cell2_03C_0.7mg.txt")
with open(echem_file, "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break  # 发现后立即退出循环，提高效率
# 读取电化学数据
echem = pd.read_csv(
    echem_file,
    sep=r"\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    parse_dates=[1, 2],
    date_format="%m/%d/%y %H:%M:%S.%f",
    index_col=None,
    engine="python",
).dropna(axis=1, how="all")
# 转换数据格式
echem["time/s"] = pd.to_datetime(echem["time/s"])
echem[["Ewe/V", "<I>/mA"]] = echem[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
# echem.info()  # noqa: ERA001

# 谱线上的时间
time_spectrum = pd.read_csv(
    Path.joinpath(path_file, r"Overview", r"Time_Index_Spectrum.csv"),
    sep=",",
    index_col=None,
    header=0,
    comment="#",
    date_format="%m/%d/%y %H:%M:%S.%f",
    parse_dates=[1],
)
time_spectrum["Time"] = pd.to_datetime(time_spectrum["Time"])
time_spectrum["ScanID"] = pd.to_numeric(time_spectrum["ScanID"])
# time_spectrum.info()  # noqa: ERA001

In [ ]:
# 提取出 P2a 的电位和电流数据
time_spectrum = time_spectrum[time_spectrum["Folder"] == r"cell2_P2b_ca"]  # cell2_P1_el , cell2_P2a_ca , cell2_P2b_ca

# 匹配谱线和电化学上的时间
echem_time = echem["time/s"].values
index_spectrum_Mn = [  # noqa: N816
    np.abs(echem_time - t).argmin() for t in time_spectrum["Time"][time_spectrum["Element"] == r"Mn"].values
]
index_spectrum_Zn = [  # noqa: N816
    np.abs(echem_time - t).argmin() for t in time_spectrum["Time"][time_spectrum["Element"] == r"Zn"].values
]

# 匹配谱线只有一条 OCV 数据， 但是时间上是有两条的时间
index_spectrum_Mn = index_spectrum_Mn[1:]  # noqa: N816
index_spectrum_Zn = index_spectrum_Zn[1:]  # noqa: N816

# 提取对应的电位和电流数据，并按电位排序
index_voltage_Mn = pd.concat(  # noqa: N816
    [
        echem.loc[index_spectrum_Mn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False),
        time_spectrum.loc[:, ["Element", "Folder"]][time_spectrum["Element"] == r"Mn"].reset_index(drop=True),
    ],
    axis=1,
    ignore_index=False,
).sort_values(by="Ewe/V", ascending=True)
index_voltage_Zn = pd.concat(  # noqa: N816
    [
        echem.loc[index_spectrum_Zn, ["Ewe/V", "<I>/mA"]].reset_index(drop=False),
        time_spectrum.loc[:, ["Element", "Folder"]][time_spectrum["Element"] == r"Zn"].reset_index(drop=True),
    ],
    axis=1,
    ignore_index=False,
).sort_values(by="Ewe/V", ascending=True)
index_voltage_Mn.reset_index(drop=False, inplace=True)
index_voltage_Zn.reset_index(drop=False, inplace=True)

# 选择需要的电化学数据以及对应的谱线
# Mn
index = [0, 9, 22, 35, 48]
selected_Mn = (  # noqa: N816
    index_voltage_Mn[index_voltage_Mn["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_Mn["index"] = pd.to_numeric(selected_Mn["index"], downcast="integer")
selected_echem_Mn = echem[echem.index <= selected_Mn["index"].iloc[-1]]  # noqa: N816
mapping_spectum_Mn = opData_Mn.loc[:, opData_Mn.columns <= selected_Mn["level_0"].iloc[-1] + 1]  # noqa: N816
mapping_spectum_Mn2 = opData2_Mn.loc[:, opData2_Mn.columns <= selected_Mn["level_0"].iloc[-1] + 1]  # noqa: N816

# Zn
index = [0, 9, 22, 35, 48]
selected_Zn = (  # noqa: N816
    index_voltage_Zn[index_voltage_Zn["level_0"].isin(index)].sort_values(by="level_0", ascending=True).reset_index(drop=True)
)
selected_Zn["index"] = pd.to_numeric(selected_Zn["index"], downcast="integer")
selected_echem_Zn = echem[echem.index <= selected_Zn["index"].iloc[-1]]  # noqa: N816
mapping_spectum_Zn = opData_Zn.loc[:, opData_Zn.columns <= selected_Zn["level_0"].iloc[-1] + 1]  # noqa: N816
mapping_spectum_Zn2 = opData2_Zn.loc[:, opData2_Zn.columns <= selected_Zn["level_0"].iloc[-1] + 1]  # noqa: N816

# 提取特定位置的值
# Mn
Intensityslice_Mn = [mapping_spectum_Mn.iloc[np.abs(mapping_spectum_Mn.iloc[:, 0] - t).argmin()] for t in [6553.09, 6561.67]]
Intensityslice_Zn = [mapping_spectum_Zn.iloc[np.abs(mapping_spectum_Zn.iloc[:, 0] - t).argmin()] for t in [9668.82]]
# Zn
Intensityslice_Mn2 = [mapping_spectum_Mn2.iloc[np.abs(mapping_spectum_Mn2.iloc[:, 0] - t).argmin()] for t in [2.40, 3.01]]
Intensityslice_Zn2 = [mapping_spectum_Zn2.iloc[np.abs(mapping_spectum_Zn2.iloc[:, 0] - t).argmin()] for t in [1.59]]

In [ ]:
# Zn satuation
index_satuation = np.arange(17, 23, 1)
mapping_spectum_Zn2.drop(index_satuation, axis=1, inplace=True)  # type: ignore
Intensityslice_Zn2[0].drop(index=index_satuation, axis=0, inplace=True)  # type: ignore

In [ ]:
# 绘图
%matplotlib inline
plt.close("all")
fig = plt.figure(figsize=(14, 5.0))
gs = gridspec.GridSpec(2, 3, width_ratios=[1, 3, 3], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

# 图 A
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(2.4)

# Mn
specical_index_Mn = [0, 9, 22, 35, 48]  # noqa: N816
ax.plot(selected_echem_Mn["Ewe/V"], selected_echem_Mn["time/s"], ls="-", lw=1.0, c=colors[0], label=r"Cell2", zorder=1)
ax.scatter(
    selected_echem_Mn["Ewe/V"].iloc[selected_Mn["index"]],
    selected_echem_Mn["time/s"].iloc[selected_Mn["index"]],
    edgecolors="face",
    c=colors[1],
    zorder=1,
)

# 特殊位置颜色标定
for idx in specical_index_Mn:
    temp = selected_Mn["index"][selected_Mn["level_0"] == idx]
    ax.scatter(
        selected_echem_Mn["Ewe/V"].iloc[temp],
        selected_echem_Mn["time/s"].iloc[temp],
        c=colors[0],
        edgecolors="face",
        marker="o",
        zorder=2,
    )

# 添加索引文本标注
for i in range(selected_Mn.shape[0]):
    temp = selected_Mn["index"].iloc[i]
    ax.text(
        selected_echem_Mn["Ewe/V"].iloc[temp] + 0.15,
        selected_echem_Mn["time/s"].iloc[temp],
        str(selected_Mn["level_0"].iloc[i]),
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
        zorder=1,
    )

# Zn
specical_index_Zn = [0, 9, 22, 35, 48]  # noqa: N816
ax.plot(selected_echem_Zn["Ewe/V"], selected_echem_Zn["time/s"], ls="-", lw=1.0, c=colors[3], label=r"Cell3", zorder=0)
ax.scatter(
    selected_echem_Zn["Ewe/V"].iloc[selected_Zn["index"]],
    selected_echem_Zn["time/s"].iloc[selected_Zn["index"]],
    edgecolors="face",
    c=colors[2],
    zorder=1,
)

# # 特殊位置颜色标定
# for idx in specical_index_Zn:
#     temp = selected_Zn['index'][selected_Zn['level_0'] == idx]  # noqa: ERA001
#     ax.scatter(selected_echem_Zn['Ewe/V'].iloc[temp], selected_echem_Zn['time/s'].iloc[temp], c=colors[2], edgecolors='face', marker='o')  # noqa: E501, ERA001

# # 添加索引文本标注
# for i in range(selected_Zn.shape[0]):
#     temp = selected_Zn['index'].iloc[i]  # noqa: ERA001
#     ax.text(selected_echem_Zn['Ewe/V'].iloc[temp] + 0.15, selected_echem_Zn['time/s'].iloc[temp],
#             str(selected_Zn['level_0'].iloc[i]), fontsize=10, verticalalignment='bottom', horizontalalignment='right', zorder=2)  # noqa: E501

ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11)
ax.set_xlim(0.8, 2.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))

# 确保时间刻度从数据最开始时间显示
ax.set_ylabel(r"Duration Time (hour)", fontsize=11, labelpad=5)
ax.set_ylim(
    selected_echem_Mn["time/s"].min() - pd.Timedelta(minutes=20),
    selected_echem_Mn["time/s"].max() + pd.Timedelta(minutes=20),
)
ax.yaxis.set_major_formatter(mdates.DateFormatter("%b-%d %H:%M"))  # 设定日期格式
ax.yaxis.set_major_locator(mdates.HourLocator(byhour=range(0, 24, 1)))
ax.yaxis.set_minor_locator(mdates.MinuteLocator(byminute=range(0, 60, 30)))
plt.yticks(rotation=0, horizontalalignment="right")
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    left=True,
    labelleft=True,
    right=False,
    labelright=False,
    top=False,
    labeltop=False,
)

ax2 = ax.twiny()
ax2.set_position((0, 0, 1, 1))
ax2.set_box_aspect(2.4)
ax2.plot(selected_echem_Mn["<I>/mA"], selected_echem_Mn["time/s"], ls="--", lw=1.0, c=colors[3], label=r"Current")  # type: ignore
ax2.set_xlabel(
    r"Current (mA)",
    fontsize=11,
)
ax2.set_xlim(-0.1, 0.1)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax2.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.05, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=True,
    labeltop=True,
    right=False,
    labelright=False,
    left=False,
    labelleft=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1])
ax = subfig.add_axes((-0.15, 0, 1, 1))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf_Mn.iloc[:, 0], pdf_Mn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[0][i], label=labels[0][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3, handletextpad=0.2)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Mn["level_0"].isin(specical_index_Mn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Mn.shape[0]):
    temp = mapping_spectum_Mn.iloc[:, mapping_spectum_Mn.columns == (selected_Mn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Mn.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Mn["level_0"].iloc[i]))

ax.set_xlim(6530, 6630)
ax.set_xlabel("Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=-10))

ax.set_ylim(0, 2.0)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 图 C
subfig = fig.add_subfigure(gs[0, 2])
ax = subfig.add_axes((-0.49, 0, 1, 1))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf_Zn.iloc[:, 0], pdf_Zn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[1][i], label=labels[1][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3, handletextpad=0.2)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Zn["level_0"].isin(specical_index_Zn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Zn.shape[0]):
    temp = mapping_spectum_Zn.iloc[:, mapping_spectum_Zn.columns == (selected_Zn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Zn.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Zn["level_0"].iloc[i]))

ax.set_xlim(9640, 9740)
ax.set_xlabel("Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=0))

ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 图 D
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, -0.45, 1.4, 1.4))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf2_Mn.iloc[:, 0], pdf2_Mn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[0][i], label=labels[0][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3, handletextpad=0.2)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Mn["level_0"].isin(specical_index_Mn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Mn.shape[0]):
    temp = mapping_spectum_Mn2.iloc[:, mapping_spectum_Mn2.columns == (selected_Mn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Mn2.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Mn["level_0"].iloc[i]))

ax.set_xlim(0, 6)
ax.set_xlabel(r"R ($\mathrm{\AA}$)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=1, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.5, offset=-10))

ax.set_ylim(0, 2.4)
ax.set_ylabel(r"FT [$\mathrm{{\chi}({\kappa})*{\kappa}{^2}}$]", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 图 E
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.03, -0.2, 0.9, 0.9))
ax.set_box_aspect(0.8)

# 标样
xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]
for i in range(2):
    ax.plot(pdf2_Zn.iloc[:, 0], pdf2_Zn.iloc[:, 1 + i], ls="--", lw=1.0, c=xascolors[1][i], label=labels[1][i])
ax.legend(loc="upper left", fontsize=10, ncol=1, bbox_to_anchor=(0.5, 1.0), handlelength=3, handletextpad=0.2)

# 选出特定的曲线， in situ 的方法
xas_colors = selected_Zn["level_0"].isin(specical_index_Zn).map(lambda x: colors[0] if x else colors[1]).to_list()
for i in range(selected_Zn.shape[0]):
    temp = mapping_spectum_Zn2.iloc[:, mapping_spectum_Zn2.columns == (selected_Zn["level_0"].iloc[i] + 1)]
    ax.plot(mapping_spectum_Zn2.iloc[:, 0], temp, ls="-", lw=1.0, c=xas_colors[i], label=str(selected_Zn["level_0"].iloc[i]))

ax.set_xlim(0, 6)
ax.set_xlabel(r"R ($\mathrm{\AA}$)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=1, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.5, offset=-10))

ax.set_ylim(0, 1.5)
ax.set_ylabel(r"FT [$\mathrm{{\chi}({\kappa})*{\kappa}{^2}}$]", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

# 图 F
subfig = fig.add_subfigure(gs[1, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((-0.4, -0.2, 0.9, 0.9))
ax.set_box_aspect(0.8)

for i in range(len(Intensityslice_Mn)):
    ax.scatter(
        np.arange(len(Intensityslice_Mn[i]) - 1),
        Intensityslice_Mn[i][1:],
        s=30,
        ls="-",
        edgecolors="face",
        lw=1,
        c=colors[0],
        label=None,
        alpha=1 - 0.3 * i,
    )
    ax.plot(
        np.arange(len(Intensityslice_Mn[i]) - 1),
        Intensityslice_Mn[i][1:],
        ls="-",
        lw=1.0,
        c=colors[0],
        label=None,
        alpha=1 - 0.3 * i,
    )

for i in range(len(Intensityslice_Zn)):
    ax.scatter(
        np.arange(len(Intensityslice_Zn[i]) - 1),
        Intensityslice_Zn[i][1:],
        s=30,
        edgecolors="face",
        ls="-",
        lw=1.0,
        c=colors[i + 2],
        label=None,
    )
    ax.plot(
        np.arange(len(Intensityslice_Zn[i]) - 1),
        Intensityslice_Zn[i][1:],
        ls="-",
        lw=1.0,
        c=colors[i + 2],
        label=None,
        alpha=1 - 0.3 * i,
    )

ax.set_xlim(0, 50)
ax.set_xlabel("Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))

ax.set_ylim(0.8, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=False,
    labelright=False,
    bottom=True,
    labelbottom=True,
    left=True,
    labelleft=True,
)

ax2 = ax.twinx()
ax2.set_position((-0.4, -0.2, 0.9, 0.9))
ax2.set_box_aspect(0.8)
for i in range(len(Intensityslice_Mn2)):
    ax2.scatter(  # type: ignore
        np.arange(len(Intensityslice_Mn2[i]) - 1),
        Intensityslice_Mn2[i][1:],
        ls="-",
        lw=1.0,
        c=colors[3],
        marker="*",
        label=None,
        s=100,
        edgecolors="face",
        alpha=1 - 0.3 * i,
    )
    ax2.plot(  # type: ignore
        np.arange(len(Intensityslice_Mn2[i]) - 1),
        Intensityslice_Mn2[i][1:],
        ls="-",
        lw=1.0,
        c=colors[3],
        label=None,
        alpha=1 - 0.3 * i,
    )
for i in range(len(Intensityslice_Zn2)):
    ax2.scatter(  # type: ignore
        np.arange(len(Intensityslice_Zn2[i]) - 1),
        Intensityslice_Zn2[i][1:],
        ls="-",
        lw=1.0,
        marker="*",
        c=colors[4 + i],
        label=None,
        s=100,
        edgecolors="face",
    )
    ax2.plot(  # type: ignore
        np.arange(len(Intensityslice_Zn2[i]) - 1),
        Intensityslice_Zn2[i][1:],
        ls="-",
        lw=1.0,
        c=colors[4 + i],
        label=None,
    )

ax2.set_ylim(0, 1.5)
ax2.set_ylabel(r"FT [$\mathrm{{\chi}({\kappa})*{\kappa}{^2}}$]", fontsize=11)
ax2.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0))
ax2.tick_params(
    axis="both",
    which="both",
    direction="out",
    labelsize=9,
    top=False,
    labeltop=False,
    right=True,
    labelright=True,
    bottom=True,
    labelbottom=True,
    left=False,
    labelleft=False,
)


# 保存图像
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P2b_1_300_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P2b_1_600_echem.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

### Details

#### P1, PCA

##### 充电数据

In [ ]:
# 读取 reference + operando 数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"  # noqa: RUF001
)

data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Mn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Mn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Mn = pd.concat([data1, data2.iloc[:, 5:]], axis=1, ignore_index=True)  # noqa: N816
charge_index = [0, 1, 2, 3, 4, *list(range(14, xas_Mn.shape[1], 1))]
xas_Mn = xas_Mn.iloc[:, charge_index]  # noqa: N816
pdf_Mn = xas_Mn.iloc[:, 0:5]  # noqa: N816
pdf_Mn.columns = [
    r"Energy",
    r"0.2M_MnSO4(aq.)",
    r"alpha_MnO2_Electrode",
    r"alpha_MnO2_Powder",
    r"FC1st",
]
opData_Mn = xas_Mn.iloc[:, [0, *list(range(5, xas_Mn.shape[1]))]]  # noqa: N816
opData_Mn.columns = list(range(0, opData_Mn.shape[1], 1))

# Zn
data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Zn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P1_Zn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Zn = pd.concat([data1, data2.iloc[:, 3:]], axis=1, ignore_index=True)  # noqa: N816
charge_index = [0, 1, 2, *list(range(12, xas_Zn.shape[1], 1))]
xas_Zn = xas_Zn.iloc[:, charge_index]  # noqa: N816

pdf_Zn = xas_Zn.iloc[:, 0:3]  # noqa: N816
pdf_Zn.columns = [r"Energy", r"0.5M_ZnSO4(aq.)", r"ZHS"]
opData_Zn = xas_Zn.iloc[:, [0, *list(range(3, xas_Zn.shape[1]))]]  # noqa: N816
opData_Zn.columns = list(range(0, opData_Zn.shape[1], 1))

# 读取 PCA 数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2\FC1st-FD2nd\Oct2022_cell2_P1\XANES"  # noqa: E501, RUF001
)
# Mn
pca_Mn = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Mn", r"PCA", r"cell2_1stFC_opXAS_P1_Mn_components.pca"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
pca_Mn2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Mn", r"PCA", r"Cumulative_variance.csv"), sep=",", index_col=None, header=0, comment="#"
)
# Zn
pca_Zn = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Zn", r"PCA", r"cell2_1stFC_opXAS_P1_Zn_components.pca"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
pca_Zn2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Zn", r"PCA", r"Cumulative_variance.csv"), sep=",", index_col=None, header=0, comment="#"
)

In [ ]:
# 剔除不好的谱线， Zn
opData_Zn = opData_Zn.drop(columns=opData_Zn.columns[14], axis=1)  # noqa: N816


In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7.0, 5))
gs = gridspec.GridSpec(2, 3, width_ratios=[1, 1, 1], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["sunset"](np.linspace(1.0, 0.0, opData_Mn.shape[1])),
    name="xas_colors",
)
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(pdf_Mn.shape[1] - 3):
    ax.plot(pdf_Mn.iloc[:, 0], pdf_Mn.iloc[:, i + 1], c=xascolors[0][i], ls="--", lw=1.0, label=labels[0][i], zorder=5)
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(opData_Mn.shape[1] - 1):
    ax.plot(opData_Mn.iloc[:, 0], opData_Mn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.05, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.45, 1),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=10,
    handlelength=3,
    handletextpad=0.2,
)
ax.text(
    0.49,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.26,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.35, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(pca_Mn.shape[1] - 1):
    (lineA,) = ax.plot(pca_Mn.iloc[:, 0], pca_Mn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0, zorder=i)  # type: ignore # noqa: N816
    line.append(lineA)
ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=-10))
ax.set_ylim(-15, 1)
ax.set_ylabel(r"PCA Intensity", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=4, offset=1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=2, offset=1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)
ax.legend(
    handles=[line[0], line[1]],
    labels=[r"Component#1", r"Component#2"],
    loc="upper left",
    bbox_to_anchor=(0.4, 0.3),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.7, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.scatter(np.arange(0, pca_Mn2.shape[0], 1), np.log10(pca_Mn2.iloc[:, 1]), s=50, c=colors[0], edgecolors="face")

ax.set_xlim(-0.5, pca_Mn2.shape[0] + 1)
ax.set_xlabel(r"Components", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(-6.0, 0.5)
ax.set_ylabel(r"log of Variance", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 D
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(0.0, 1.0, opData_Zn.shape[1])),
    name="xas_colors",
)
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]

for i in range(pdf_Zn.shape[1] - 1):
    ax.plot(pdf_Zn.iloc[:, 0], pdf_Zn.iloc[:, i + 1], c=xascolors[1][i], ls="--", lw=1.0, label=labels[1][i], zorder=5)
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(opData_Zn.shape[1] - 1):
    ax.plot(opData_Zn.iloc[:, 0], opData_Zn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0)  # type: ignore

ax.set_xlim(9650, 9750)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.05, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.45, 1),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=10,
    handlelength=3,
    handletextpad=0.2,
)
ax.text(
    0.49,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.26,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

# 图 E
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.35, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(pca_Zn.shape[1] - 1):
    (lineA,) = ax.plot(pca_Zn.iloc[:, 0], pca_Zn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0, zorder=i)  # type: ignore # noqa: N816
    line.append(lineA)
ax.set_xlim(9650, 9750)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=-10))
ax.set_ylim(-15.0, 1)
ax.set_ylabel(r"PCA Intensity", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=4, offset=1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=2, offset=1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)
ax.legend(
    handles=[line[0], line[1]],
    labels=[r"Component#1", r"Component#2"],
    loc="upper left",
    bbox_to_anchor=(0.4, 0.3),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)

# 图 F
subfig = fig.add_subfigure(gs[1, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.7, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.scatter(np.arange(0, pca_Zn2.shape[0], 1), np.log10(pca_Zn2.iloc[:, 1]), s=50, c=colors[1], edgecolors="face")

ax.set_xlim(-0.5, pca_Zn2.shape[0] + 1)
ax.set_xlabel(r"Components", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(-8, 0.5)
ax.set_ylabel(r"log of Variance", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P1_3_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P1_3_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### P2a, PCA

##### 充电数据

In [ ]:
# 读取 reference + operando 数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"  # noqa: RUF001
)

data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Mn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Mn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Mn = pd.concat([data1, data2.iloc[:, 5:]], axis=1, ignore_index=True)  # noqa: N816
charge_index = [0, 1, 2, 3, 4, *list(range(14, xas_Mn.shape[1], 1))]
xas_Mn = xas_Mn.iloc[:, charge_index]  # noqa: N816
pdf_Mn = xas_Mn.iloc[:, 0:5]  # noqa: N816
pdf_Mn.columns = [
    r"Energy",
    r"0.2M_MnSO4(aq.)",
    r"alpha_MnO2_Electrode",
    r"alpha_MnO2_Powder",
    r"FC1st",
]
opData_Mn = xas_Mn.iloc[:, [0, *list(range(5, xas_Mn.shape[1]))]]  # noqa: N816
opData_Mn.columns = list(range(0, opData_Mn.shape[1], 1))

# Zn
data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Zn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Zn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Zn = pd.concat([data1, data2.iloc[:, 3:]], axis=1, ignore_index=True)  # noqa: N816
charge_index = [0, 1, 2, *list(range(12, xas_Zn.shape[1], 1))]
xas_Zn = xas_Zn.iloc[:, charge_index]  # noqa: N816

pdf_Zn = xas_Zn.iloc[:, 0:3]  # noqa: N816
pdf_Zn.columns = [r"Energy", r"0.5M_ZnSO4(aq.)", r"ZHS"]
opData_Zn = xas_Zn.iloc[:, [0, *list(range(3, xas_Zn.shape[1]))]]  # noqa: N816
opData_Zn.columns = list(range(0, opData_Zn.shape[1], 1))

# 读取 PCA 数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2\FC1st-FD2nd\Oct2022_cell2_P2a\XANES"  # noqa: E501, RUF001
)
# Mn
pca_Mn = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Mn", r"PCA", r"cell2_1stFC_opXAS_P2a_Mn_components.pca"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
pca_Mn2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Mn", r"PCA", r"Cumulative_variance.csv"), sep=",", index_col=None, header=0, comment="#"
)
# Zn
pca_Zn = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Zn", r"PCA", r"cell2_1stFC_opXAS_P2a_Zn_components.pca"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
pca_Zn2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Zn", r"PCA", r"Cumulative_variance.csv"), sep=",", index_col=None, header=0, comment="#"
)

In [ ]:
# 提出不好的噪点
opData_Mn = opData_Mn.drop(columns=opData_Mn.columns[13], axis=1)  # noqa: N816


In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7.0, 5))
gs = gridspec.GridSpec(2, 3, width_ratios=[1, 1, 1], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["sunset"](np.linspace(1.0, 0.0, opData_Mn.shape[1])),
    name="xas_colors",
)
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(pdf_Mn.shape[1] - 3):
    ax.plot(pdf_Mn.iloc[:, 0], pdf_Mn.iloc[:, i + 1], c=xascolors[0][i], ls="--", lw=1.0, label=labels[0][i], zorder=5)
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(opData_Mn.shape[1] - 1):
    ax.plot(opData_Mn.iloc[:, 0], opData_Mn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.05, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.45, 1),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=10,
    handlelength=3,
    handletextpad=0.2,
)
ax.text(
    0.49,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.26,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.35, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(pca_Mn.shape[1] - 1):
    (lineA,) = ax.plot(pca_Mn.iloc[:, 0], pca_Mn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0, zorder=i)  # type: ignore # noqa: N816
    line.append(lineA)
ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=-10))
ax.set_ylim(-10, 2.0)
ax.set_ylabel(r"PCA Intensity", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)
ax.legend(
    handles=[line[0], line[1], line[2]],
    labels=[r"Component#1", r"Component#2", r"Component#3"],
    loc="upper left",
    bbox_to_anchor=(0.4, 0.33),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.7, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.scatter(np.arange(0, pca_Mn2.shape[0], 1), np.log10(pca_Mn2.iloc[:, 1]), s=50, c=colors[0], edgecolors="face")

ax.set_xlim(-0.5, pca_Mn2.shape[0] + 1)
ax.set_xlabel(r"Components", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(-8.0, 0.5)
ax.set_ylabel(r"log of Variance", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 D
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(0.0, 1.0, opData_Zn.shape[1])),
    name="xas_colors",
)
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]

for i in range(pdf_Zn.shape[1] - 1):
    ax.plot(pdf_Zn.iloc[:, 0], pdf_Zn.iloc[:, i + 1], c=xascolors[1][i], ls="--", lw=1.0, label=labels[1][i], zorder=5)
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(opData_Zn.shape[1] - 1):
    ax.plot(opData_Zn.iloc[:, 0], opData_Zn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0)  # type: ignore

ax.set_xlim(9650, 9750)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.05, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.45, 1),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=10,
    handlelength=3,
    handletextpad=0.2,
)
ax.text(
    0.49,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.26,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

# 图 E
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.35, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(pca_Zn.shape[1] - 1):
    (lineA,) = ax.plot(pca_Zn.iloc[:, 0], pca_Zn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0, zorder=i)  # type: ignore # noqa: N816
    line.append(lineA)
ax.set_xlim(9650, 9750)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=-10))
ax.set_ylim(-15, 1)
ax.set_ylabel(r"PCA Intensity", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=4, offset=1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=2, offset=1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)
ax.legend(
    handles=[line[0], line[1], line[2]],
    labels=[r"Component#1", r"Component#2", r"Component#3"],
    loc="upper left",
    bbox_to_anchor=(0.4, 0.35),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)

# 图 F
subfig = fig.add_subfigure(gs[1, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.7, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.scatter(np.arange(0, pca_Zn2.shape[0], 1), np.log10(pca_Zn2.iloc[:, 1]), s=50, c=colors[1], edgecolors="face")

ax.set_xlim(-0.5, pca_Zn2.shape[0] + 1)
ax.set_xlabel(r"Components", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(-8, 0.5)
ax.set_ylabel(r"log of Variance", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P2a_3_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P2a_3_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### P2b, PCA

##### 充电数据

In [ ]:
# 读取 reference + operando 数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"  # noqa: RUF001
)

data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Mn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Mn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Mn = pd.concat([data1, data2.iloc[:, 5:]], axis=1, ignore_index=True)  # noqa: N816
charge_index = [0, 1, 2, 3, 4, *list(range(14, xas_Mn.shape[1], 1))]
xas_Mn = xas_Mn.iloc[:, charge_index]  # noqa: N816
pdf_Mn = xas_Mn.iloc[:, 0:5]  # noqa: N816
pdf_Mn.columns = [
    r"Energy",
    r"0.2M_MnSO4(aq.)",
    r"alpha_MnO2_Electrode",
    r"alpha_MnO2_Powder",
    r"FC1st",
]
opData_Mn = xas_Mn.iloc[:, [0, *list(range(5, xas_Mn.shape[1]))]]  # noqa: N816
opData_Mn.columns = list(range(0, opData_Mn.shape[1], 1))

# Zn
data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Zn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Zn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Zn = pd.concat([data1, data2.iloc[:, 3:]], axis=1, ignore_index=True)  # noqa: N816
charge_index = [0, 1, 2, *list(range(12, xas_Zn.shape[1], 1))]
xas_Zn = xas_Zn.iloc[:, charge_index]  # noqa: N816

pdf_Zn = xas_Zn.iloc[:, 0:3]  # noqa: N816
pdf_Zn.columns = [r"Energy", r"0.5M_ZnSO4(aq.)", r"ZHS"]
opData_Zn = xas_Zn.iloc[:, [0, *list(range(3, xas_Zn.shape[1]))]]  # noqa: N816
opData_Zn.columns = list(range(0, opData_Zn.shape[1], 1))

# 读取 PCA 数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2\FC1st-FD2nd\Oct2022_cell3_P2b\XANES"  # noqa: E501, RUF001
)
# Mn
pca_Mn = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Mn", r"PCA", r"cell2_1stFC_opXAS_P2b_Mn_components.pca"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
pca_Mn2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Mn", r"PCA", r"Cumulative_variance.csv"), sep=",", index_col=None, header=0, comment="#"
)
# Zn
pca_Zn = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Zn", r"PCA", r"cell2_1stFC_opXAS_P2b_Zn_components.pca"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
pca_Zn2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Zn", r"PCA", r"Cumulative_variance.csv"), sep=",", index_col=None, header=0, comment="#"
)

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7.0, 5))
gs = gridspec.GridSpec(2, 3, width_ratios=[1, 1, 1], height_ratios=[1, 1], wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["sunset"](np.linspace(1.0, 0.0, opData_Mn.shape[1])),
    name="xas_colors",
)
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(pdf_Mn.shape[1] - 3):
    ax.plot(pdf_Mn.iloc[:, 0], pdf_Mn.iloc[:, i + 1], c=xascolors[0][i], ls="--", lw=1.0, label=labels[0][i], zorder=5)
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(opData_Mn.shape[1] - 1):
    ax.plot(opData_Mn.iloc[:, 0], opData_Mn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.05, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.45, 1),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=10,
    handlelength=3,
    handletextpad=0.2,
)
ax.text(
    0.49,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.26,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.35, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(pca_Mn.shape[1] - 1):
    (lineA,) = ax.plot(pca_Mn.iloc[:, 0], pca_Mn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0, zorder=i)  # type: ignore # noqa: N816
    line.append(lineA)
ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=-10))
ax.set_ylim(-10, 2)
ax.set_ylabel(r"PCA Intensity", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=-1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=-1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)
ax.legend(
    handles=[line[0], line[1], line[2]],
    labels=[r"Component#1", r"Component#2", r"Component#3"],
    loc="upper left",
    bbox_to_anchor=(0.4, 0.33),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.7, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.scatter(np.arange(0, pca_Mn2.shape[0], 1), np.log10(pca_Mn2.iloc[:, 1]), s=50, c=colors[0], edgecolors="face")

ax.set_xlim(-0.5, pca_Mn2.shape[0] + 1)
ax.set_xlabel(r"Components", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(-8.0, 0.5)
ax.set_ylabel(r"log of Variance", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 D
subfig = fig.add_subfigure(gs[1, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(0.0, 1.0, opData_Zn.shape[1])),
    name="xas_colors",
)
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]

for i in range(pdf_Zn.shape[1] - 1):
    ax.plot(pdf_Zn.iloc[:, 0], pdf_Zn.iloc[:, i + 1], c=xascolors[1][i], ls="--", lw=1.0, label=labels[1][i], zorder=5)
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(opData_Zn.shape[1] - 1):
    ax.plot(opData_Zn.iloc[:, 0], opData_Zn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0)  # type: ignore

ax.set_xlim(9650, 9750)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.05, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])
ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.45, 1),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=10,
    handlelength=3,
    handletextpad=0.2,
)
ax.text(
    0.49,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.26,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.18,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

# 图 E
subfig = fig.add_subfigure(gs[1, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.35, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(pca_Zn.shape[1] - 1):
    (lineA,) = ax.plot(pca_Zn.iloc[:, 0], pca_Zn.iloc[:, i + 1], c=xas_colors.colors[i], ls="-", lw=1.0, zorder=i)  # type: ignore # noqa: N816
    line.append(lineA)
ax.set_xlim(9650, 9750)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=-10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=-10))
ax.set_ylim(-15, 1.0)
ax.set_ylabel(r"PCA Intensity", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=3, offset=1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=1.5, offset=1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)
ax.legend(
    handles=[line[0], line[1], line[2]],
    labels=[r"Component#1", r"Component#2", r"Component#3"],
    loc="upper left",
    bbox_to_anchor=(0.4, 0.35),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)

# 图 F
subfig = fig.add_subfigure(gs[1, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.7, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

ax.scatter(np.arange(0, pca_Zn2.shape[0], 1), np.log10(pca_Zn2.iloc[:, 1]), s=50, c=colors[1], edgecolors="face")

ax.set_xlim(-0.5, pca_Zn2.shape[0] + 1)
ax.set_xlabel(r"Components", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(-8, 0.5)
ax.set_ylabel(r"log of Variance", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.5))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P2b_3_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"opXAS_2023_cell2_P2b_3_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### P2a MCR-ALS

##### Mn，充电

In [ ]:
# 读取 reference + operando 数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"  # noqa: RUF001
)

data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Mn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Mn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Mn = pd.concat([data1, data2.iloc[:, 5:]], axis=1, ignore_index=True)  # noqa: N816
charge_index = [0, 1, 2, 3, 4, *list(range(14, xas_Mn.shape[1], 1))]
xas_Mn = xas_Mn.iloc[:, charge_index]  # noqa: N816
pdf_Mn = xas_Mn.iloc[:, 0:5]  # noqa: N816
pdf_Mn.columns = [
    r"Energy",
    r"0.2M_MnSO4(aq.)",
    r"alpha_MnO2_Electrode",
    r"alpha_MnO2_Powder",
    r"FC1st",
]
opData_Mn = xas_Mn.iloc[:, [0, *list(range(5, xas_Mn.shape[1]))]]  # noqa: N816
opData_Mn.columns = list(range(0, opData_Mn.shape[1], 1))

In [ ]:
# 剔除 Mn 中的噪点
opData_Mn = opData_Mn.drop(opData_Mn.columns[13], axis=1)  # noqa: N816

In [ ]:
Mn_NormalData = xr.Dataset(
    data_vars={
        "Spectrum": (["energy", "spectrum_number"], opData_Mn.iloc[:, 1:]),
        "Reference": (["energy", "reference_number"], pdf_Mn.iloc[:, [1, 2, 4]]),
    },
    coords={
        "spectrum_number": list(range(0, opData_Mn.shape[1] - 1, 1)),
        "energy": opData_Mn.iloc[:, 0],
        "reference_number": [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1st"],
    },
)

In [ ]:
data = scp.NDDataset(
    data=Mn_NormalData["Spectrum"].data.T,
    name="opXAS_Mn_CLAESS_2022",
    author="Cheng Liu, Ashely, and Dino Tonti",
    description="opXAS_Mn_CLAESS_2022",
    history="creation",
    title="absorption",
    units=ur.absorbance,
)

data.y = scp.Coord.arange(Mn_NormalData["Spectrum"].shape[1], title="sperctum number")
data.y.labels = Mn_NormalData.coords["spectrum_number"]
data.x = scp.Coord(Mn_NormalData.coords["energy"], title="energy", units=ur.eV)


ref = scp.NDDataset(data=Mn_NormalData["Reference"].data.T, name="ref_NOTOs_2024", units=ur.absorbance)
ref.y = scp.Coord.arange(Mn_NormalData["Reference"].shape[1], title="sperctum number")
ref.x = scp.Coord(Mn_NormalData.coords["energy"], title="energy", units=ur.eV)

_ = data[:, 6530.0:6630.0].plot(title="Normalized Data")
_ = ref[:, 6530.0:6630.0].plot(title=r"Normalized Data and Refs", ls="--", clear=False)

XANES = data[:, 6530.0:6630.0]
ref_XANES = ref[:, 6530.0:6630.0]  # noqa: N816

##### PCA

In [ ]:
# # pca analysis
pca = scp.PCA()
pca.fit(XANES)
pca.printev()
# _ = pca.screeplot()  # noqa: ERA001
# loadings = pca.loadings  # noqa: ERA001
# scores = pca.scores  # noqa: ERA001
# _ = pca.loadings.plot(); _.set_ylabel('loadings')  # noqa: ERA001
# _ = pca.scores.T.plot(); _.set_ylabel('scores')  # noqa: ERA001
# _ =pca.scoreplot(pca.scores, 1, 2, color_mapping="labels")  # noqa: ERA001

In [ ]:
Mn_PCA = xr.Dataset(
    data_vars={
        "Explained_variance_ratio": (["spectrum_number"], pca.ev_ratio),
        "Cumulative_explained_variance": (["spectrum_number"], pca.ev_cum),
        "PCA_recon_Data": (["spectrum_number", "pca_energy"], pca.inverse_transform().data),
        "PCA_residual": (["spectrum_number", "pca_energy"], XANES.data - pca.inverse_transform().data),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
    },
)

##### Components = 2

##### 方案一，两个标样做约束

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]
    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr = scp.MCRALS(
    max_iter=300,
    maxdiv=100,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr.hardSpec = [0, 1]
mcr.getSpec = get_St
mcr.hardSt_to_St_idx = [0, 1]
mcr.fit(XANES, St0)
mcr.kwargsGetConc = {"ivp_solver_kwargs": {"return_NDDataset": False}, "optimizer_kwargs": {"options": {"disp": False}}}

_ = mcr.C.T.plot(clear=True, title=r"Concentration")
_ = mcr.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--", title=r"MCR-ALS and Refs")

_ = mcr.plotmerit(nb_traces=5, clear=True)
mcr_residual = XANES - mcr.inverse_transform()

In [ ]:
Mn_MCR_2 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr.St.shape[0]),
    },
)

(xr.merge([Mn_NormalData, Mn_PCA, Mn_MCR_2], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2a_Mn_MCR_2.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCR_2["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1st"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCR_2[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCR_2[r"spectrum_number"],
        Mn_MCR_2[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.4, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Mn_MCR_2[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.1, 0.9)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=labels[0][i],
        zorder=5,
    )
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(Mn_MCR_2["MCR_HM_St"].shape[0]):
    ax.plot(Mn_MCR_2["MCR_HM_St"]["pca_energy"], Mn_MCR_2["MCR_HM_St"][i, :], c=xascolors[0][i], label=None, ls="-", lw=1.0)

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCR_2["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCR_2["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCR_2["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCR_2["MCR_HM_residual"]["pca_energy"],
        Mn_MCR_2["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCR_2_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCR_2_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### 方案二，Hard-Model (A --> B), 利用 LCF, Mn2+, 0.547 和 aMnO2, 0.453 或者 Mn2+, 0.515 和 FC1st, 0.485

In [ ]:
# kinetic model
# A，用的是 Mn2+ + PRISTINE
from tqdm.notebook import tqdm

reactions = ("A -> B", "B -> A")  # note the coma: this is now a tuple of size 1, 不管方程式的系数：这个从 Zn 那边验证过
species_concentrations = {"A": 0.547, "B": 0.453}  # A = Mn2+, B = MnO2
results = []


def evaluate_kinetics(k, XANES, ref_XANES):  # noqa: N803
    try:
        k0 = np.array([k, -k])  # as we have a single reaction, we must use a single rate constant.
        kin = scp.ActionMassKinetics(reactions, species_concentrations, k0)
        Ckin = kin.integrate(XANES.y.data)  # noqa: N806

        param_to_optimize = {
            "k[0]": k,
            "k[1]": -k,
        }
        mcr_2 = scp.MCRALS(
            max_iter=100,
            maxdiv=10,
            log_level="ERROR",
            tol=0.001,
            closureConc=[0, 1],
            closureMethod="constantSum",
            closureTarget="default",  # closure constraints
            nonnegConc="all",
            solverConc="nnls",  # Non-negativity constraint
            nonnegSpec="all",
            solverSpec="nnls",  # Non-negativity constraint
        )
        mcr_2.hardConc = [0, 1]
        mcr_2.getConc = kin.fit_to_concentrations
        mcr_2.argsGetConc = ([0, 1], [0, 1], param_to_optimize)
        mcr_2.kwargsGetConc = {
            "ivp_solver_kwargs": {"return_NDDataset": False},
            "optimizer_kwargs": {"options": {"disp": False}},
        }
        mcr_2.fit(XANES, Ckin)
        # std_st_Mn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1) # Mn2+  # noqa: ERA001
        # return [k, *std_st_Mn], mcr_2  # noqa: ERA001
        std_st_Mn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1)  # Mn2+  # noqa: N806
        std_st_Mn2 = np.std(np.asarray(mcr_2.St.data - ref_XANES[1].data), axis=1)  # MnO2  # noqa: N806
        std_Mn = [std_st_Mn[i] + std_st_Mn2[j] for i in range(len(std_st_Mn)) for j in range(len(std_st_Mn2)) if i != j]  # noqa: N806
        return [k, std_Mn[0], std_Mn[1]], mcr_2
    except Exception as e:
        print(f"[Warning] k={k:.3f}, failed: {e}")
        return None


for k in tqdm(np.linspace(0.0, 0.5, 500, endpoint=True)):
    result_row = evaluate_kinetics(k, XANES, ref_XANES)
    if result_row[0]:  # type: ignore
        results.append(result_row[0])  # type: ignore

result = pd.DataFrame(results, columns=["k", "std_comp1", "std_comp2"])

# 打印最小标准差对应的参数组合
for i in range(1, 3):
    print(result.iloc[result.iloc[:, i].idxmin(), :])  # type: ignore

# 保存结果
result.to_csv(
    Path.joinpath(path_out, r"P2a_Mn_MCRHM_1reaction_comp_2_A_B.csv"),
    sep=",",
    index=False,
    header=True,
)

In [ ]:
# 将上述结果作为初始值，进行最后的结果
#  kinetic model
ka = 0.006012
reactions = ("A -> B", "B -> A")  # note the coma: this is now a tuple of size 1
species_concentrations = {"A": 0.547, "B": 0.453}  # A = Mn2+, B = MnO2

mcr_2 = evaluate_kinetics(ka, XANES, ref_XANES)[1]  # type: ignore
_ = mcr_2.C.T.plot(ylabel=r"relative ratio")
_ = mcr_2.C_constrained.T.plot(clear=False)
_ = mcr_2.St.plot(clear=True, ylims=(-0.1, 1.8))
_ = ref_XANES.plot(clear=False, ls="--")

_ = mcr_2.plotmerit(nb_traces=5)
mcr_2_residual = XANES - mcr_2.inverse_transform()

Mn_MCRHM_OnereactionData = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_2.C.data),
        "MCR_HM_C_constrained": (["spectrum_number", "mcr_number"], mcr_2.C_constrained.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_2.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_2.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_2_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_2.St.shape[0]),
    },
    attrs={"reactions": reactions, "species_concentrations": str(species_concentrations), "k0": ka},
)

(
    xr.merge([Mn_NormalData, Mn_PCA, Mn_MCRHM_OnereactionData], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(
        Path.joinpath(path_out, r"P2a_Mn_MCRHM_1reaction_comp_2_A_B.NETCDF4"), mode="w", engine="h5netcdf"
    )
)

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCRHM_OnereactionDataSS["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1st"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCRHM_OnereactionData[r"spectrum_number"],
        Mn_MCRHM_OnereactionData[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.3, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

line = []
for i in range(Mn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"].shape[1]):
    (lineB,) = ax.plot(  # noqa: N816
        Mn_MCRHM_OnereactionData[r"spectrum_number"],
        Mn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"][:, i],
        c="b",
        label=labels[0][i],
        ls="--",
    )
    line.append(lineB)
ax.legend(
    handles=[line[0]],
    labels=[r"Initial Concentration"],
    loc="upper left",
    bbox_to_anchor=(0.3, 0.15),
    ncols=1,
    handlelength=3,
    labelcolor="linecolor",
    fontsize=9,
)

ax.set_xlim(-0.5, Mn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=labels[0][i],
        zorder=5,
    )
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(Mn_MCRHM_OnereactionData["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCRHM_OnereactionData["MCR_HM_St"]["pca_energy"],
        Mn_MCRHM_OnereactionData["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=None,
        ls="-",
        lw=1.0,
    )

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCRHM_OnereactionData["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCRHM_OnereactionData["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCRHM_OnereactionData["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCRHM_OnereactionData["MCR_HM_residual"]["pca_energy"],
        Mn_MCRHM_OnereactionData["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCRHM_1reaction_comp_2_A_B_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCRHM_1reaction_comp_2_A_B_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

In [ ]:
# kinetic model
# B，用的是 Mn2+ + FC1st
from tqdm.notebook import tqdm

reactions = ("A -> B",)  # note the coma: this is now a tuple of size 1, 不管方程式的系数：这个从 Zn 那边验证过
species_concentrations = {"A": 0.515, "B": 0.485}  # A = Mn2+, B = FC1st
results = []


def evaluate_kinetics(k, XANES, ref_XANES):  # noqa: N803
    try:
        k0 = np.array([
            k,
        ])  # as we have a single reaction, we must use a single rate constant.
        kin = scp.ActionMassKinetics(reactions, species_concentrations, k0)
        Ckin = kin.integrate(XANES.y.data)  # noqa: N806

        param_to_optimize = {
            "k[0]": k,
        }
        mcr_2 = scp.MCRALS(
            max_iter=100,
            maxdiv=10,
            log_level="ERROR",
            tol=0.001,
            closureConc=[0, 1],
            closureMethod="constantSum",
            closureTarget="default",  # closure constraints
            nonnegConc="all",
            solverConc="nnls",  # Non-negativity constraint
            nonnegSpec="all",
            solverSpec="nnls",  # Non-negativity constraint
        )
        mcr_2.hardConc = [0, 1]
        mcr_2.getConc = kin.fit_to_concentrations
        mcr_2.argsGetConc = ([0, 1], [0, 1], param_to_optimize)
        mcr_2.kwargsGetConc = {
            "ivp_solver_kwargs": {"return_NDDataset": False},
            "optimizer_kwargs": {"options": {"disp": False}},
        }
        mcr_2.fit(XANES, Ckin)
        # std_st_Mn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1) # Mn2+  # noqa: ERA001
        # return [k, std_st_Mn[0], std_st_Mn[1]], mcr_2  # noqa: ERA001
        std_st_Mn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1)  # Mn2+  # noqa: N806
        std_st_Mn2 = np.std(np.asarray(mcr_2.St.data - ref_XANES[2].data), axis=1)  # MnO2  # noqa: N806
        std_Mn = [std_st_Mn[i] + std_st_Mn2[j] for i in range(len(std_st_Mn)) for j in range(len(std_st_Mn2)) if i != j]  # noqa: N806
        return [k, std_Mn[0], std_Mn[1]], mcr_2
    except Exception as e:
        print(f"[Warning] k={k:.3f}, failed: {e}")
        return None


for k in tqdm(np.linspace(0.0, 0.5, 500, endpoint=True)):
    result_row = evaluate_kinetics(k, XANES, ref_XANES)
    if result_row[0]:  # type: ignore
        results.append(result_row[0])  # type: ignore

result = pd.DataFrame(results, columns=["k", "std_comp1", "std_comp2"])

# 打印最小标准差对应的参数组合
for i in range(1, 3):
    print(result.iloc[result.iloc[:, i].idxmin(), :])  # type: ignore

# 保存结果
result.to_csv(
    Path.joinpath(path_out, r"P2a_Mn_MCRHM_1reaction_comp_2_A_B_FC1st_Mn2+.csv"),
    sep=",",
    index=False,
    header=True,
)

In [ ]:
# 将上述结果作为初始值，进行最后的结果
#  kinetic model
ka = 0.015030
reactions = ("A -> B",)  # note the coma: this is now a tuple of size 1
species_concentrations = {"A": 0.515, "B": 0.485}  # A = Mn2+, B = FC1st

mcr_2 = evaluate_kinetics(ka, XANES, ref_XANES)[1]  # type: ignore
_ = mcr_2.C.T.plot(ylabel=r"relative ratio")
_ = mcr_2.C_constrained.T.plot(clear=False)
_ = mcr_2.St.plot(clear=True, ylims=(-0.1, 1.8))
_ = ref_XANES.plot(clear=False, ls="--")

_ = mcr_2.plotmerit(nb_traces=5)
mcr_2_residual = XANES - mcr_2.inverse_transform()

Mn_MCRHM_OnereactionData = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_2.C.data),
        "MCR_HM_C_constrained": (["spectrum_number", "mcr_number"], mcr_2.C_constrained.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_2.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_2.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_2_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_2.St.shape[0]),
    },
    attrs={"reactions": reactions, "species_concentrations": str(species_concentrations), "k0": ka},
)

(
    xr.merge([Mn_NormalData, Mn_PCA, Mn_MCRHM_OnereactionData], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(
        Path.joinpath(path_out, r"P2a_Mn_MCRHM_1reaction_comp_2_A_B_FC1st_Mn2+.NETCDF4"), mode="w", engine="h5netcdf"
    )
)

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCRHM_OnereactionData["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1st"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCRHM_OnereactionData[r"spectrum_number"],
        Mn_MCRHM_OnereactionData[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.1, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

line = []
for i in range(Mn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"].shape[1]):
    (lineB,) = ax.plot(  # noqa: N816
        Mn_MCRHM_OnereactionData[r"spectrum_number"],
        Mn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"][:, i],
        c="b",
        label=labels[0][i],
        ls="--",
    )
    line.append(lineB)
ax.legend(
    handles=[line[0]],
    labels=[r"Initial Concentration"],
    loc="upper left",
    bbox_to_anchor=(0.3, 0.15),
    ncols=1,
    handlelength=3,
    labelcolor="linecolor",
    fontsize=9,
)

ax.set_xlim(-0.5, Mn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.1, 0.8)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=labels[0][i],
        zorder=5,
    )
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(Mn_MCRHM_OnereactionData["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCRHM_OnereactionData["MCR_HM_St"]["pca_energy"],
        Mn_MCRHM_OnereactionData["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=None,
        ls="-",
        lw=1.0,
    )

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCR_2["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCR_2["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCR_2["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCR_2["MCR_HM_residual"]["pca_energy"],
        Mn_MCR_2["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCRHM_1reaction_comp_2_A_B_FC1st_Mn2+_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCRHM_1reaction_comp_2_A_B_FC1st_Mn2+_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### Components = 3

##### 方案一，三个标样做约束， Mn2+，MnO2，FullCharge

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]
    St[2, :] = ref_XANES[2]
    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
        ref_XANES[2].squeeze().data,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr_3 = scp.MCRALS(
    max_iter=300,
    maxdiv=100,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1, 2],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr_3.hardSpec = [0, 1, 2]
mcr_3.getSpec = get_St
mcr_3.hardSt_to_St_idx = [0, 1, 2]
mcr_3.fit(XANES, St0)
mcr_3.kwargsGetConc = {
    "ivp_solver_kwargs": {"return_NDDataset": False},
    "optimizer_kwargs": {"options": {"disp": False}},
}

_ = mcr_3.C.T.plot(clear=True, title=r"Concentration")
_ = mcr_3.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--", title=r"MCR-ALS and Refs")

_ = mcr_3.plotmerit(nb_traces=5, clear=True)
mcr_3_residual = XANES - mcr_3.inverse_transform()

In [ ]:
Mn_MCR_3 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_3.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_3.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_3.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_3_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_3.St.shape[0]),
    },
)

(xr.merge([Mn_NormalData, Mn_PCA, Mn_MCR_3], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2a_Mn_MCR_3.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCR_3["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1st"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCR_3[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCR_3[r"spectrum_number"],
        Mn_MCR_3[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.45, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Mn_MCR_3[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)


for i in range(Mn_MCR_3["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_St"]["pca_energy"],
        Mn_MCR_3["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCR_3["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCR_3["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCR_3["MCR_HM_residual"]["pca_energy"],
        Mn_MCR_3["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCR_3_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCR_3_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()


##### 方案二，三个标样做约束， Mn2+，MnO2，Dino 从 FullCharge 分离出来的

In [ ]:
from scipy.interpolate import interp1d

path_dino = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2\Dino")  # noqa: E501, RUF001
dino = pd.read_csv(Path.joinpath(path_dino, r"FC1st_3rd_phase.txt"), sep=r"\s+", header=None, index_col=None)
dino = dino[(dino.iloc[:, 0] >= 6530.0) & (dino.iloc[:, 0] <= 6630.0)].copy()

interp_func = interp1d(dino.iloc[:, 0], dino.iloc[:, 3], kind="quadratic", bounds_error=False, fill_value=0)
dino_interp = interp_func(ref_XANES.x.data)

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]
    St[2, :] = dino_interp
    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
        dino_interp,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr_3 = scp.MCRALS(
    max_iter=300,
    maxdiv=100,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1, 2],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr_3.hardSpec = [0, 1, 2]
mcr_3.getSpec = get_St
mcr_3.hardSt_to_St_idx = [0, 1, 2]
mcr_3.fit(XANES, St0)
mcr_3.kwargsGetConc = {
    "ivp_solver_kwargs": {"return_NDDataset": False},
    "optimizer_kwargs": {"options": {"disp": False}},
}

_ = mcr_3.C.T.plot(clear=True, title=r"Concentration")
_ = mcr_3.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--", title=r"MCR-ALS and Refs")

_ = mcr_3.plotmerit(nb_traces=5, clear=True)
mcr_3_residual = XANES - mcr_3.inverse_transform()

In [ ]:
Mn_MCR_3 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_3.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_3.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_3.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_3_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_3.St.shape[0]),
    },
)

(xr.merge([Mn_NormalData, Mn_PCA, Mn_MCR_3], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2a_Mn_MCR_3_FC.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCR_3["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1stDino"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCR_3[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCR_3[r"spectrum_number"],
        Mn_MCR_3[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.4, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Mn_MCR_3[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)


for i in range(Mn_MCR_3["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_St"]["pca_energy"],
        Mn_MCR_3["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.4, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCR_3["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCR_3["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCR_3["MCR_HM_residual"]["pca_energy"],
        Mn_MCR_3["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCR_3_FC_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCR_3_FC_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### 方案三，两个标样做约束， Mn2+，MnO2

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]

    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
        ref_XANES[2].squeeze().data,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr_3 = scp.MCRALS(
    max_iter=300,
    maxdiv=100,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1, 2],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr_3.hardSpec = [0, 1, 2]
mcr_3.getSpec = get_St
mcr_3.hardSt_to_St_idx = [0, 1, 2]
mcr_3.fit(XANES, St0)
mcr_3.kwargsGetConc = {
    "ivp_solver_kwargs": {"return_NDDataset": False},
    "optimizer_kwargs": {"options": {"disp": False}},
}

_ = mcr_3.C.T.plot(clear=True, title=r"Concentration")
_ = mcr_3.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--", title=r"MCR-ALS and Refs")

_ = mcr_3.plotmerit(nb_traces=5, clear=True)
mcr_3_residual = XANES - mcr_3.inverse_transform()

In [ ]:
Mn_MCR_3 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_3.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_3.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_3.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_3_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_3.St.shape[0]),
    },
)

(xr.merge([Mn_NormalData, Mn_PCA, Mn_MCR_3], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2a_Mn_MCR_3_2.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCR_3["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"#3"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCR_3[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCR_3[r"spectrum_number"],
        Mn_MCR_3[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.1, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Mn_MCR_3[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)


for i in range(Mn_MCR_3["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_St"]["pca_energy"],
        Mn_MCR_3["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCR_3["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCR_3["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCR_3["MCR_HM_residual"]["pca_energy"],
        Mn_MCR_3["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCR_3_2_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCR_3_2_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### 方案四，Hard-Model ("A -> B", "B + A -> C"), 利用 CLAESS Refers 确定初始浓度值，-20 到 80 eV, 0.547 和 0.453

In [ ]:
from tqdm.notebook import tqdm

reactions = ("A -> B", "A + B -> C")
species_concentrations = {"A": 0.547, "B": 0, "C": 0.453}
results = []


def evaluate_kinetics_2(k, m, XANES, ref_XANES):  # noqa: N803
    try:
        k0 = np.array([k, m])
        kin = scp.ActionMassKinetics(reactions, species_concentrations, k0)
        Ckin = kin.integrate(XANES.y.data)  # noqa: N806

        param_to_optimize = {"k[0]": k, "k[1]": m}
        mcr_4 = scp.MCRALS(
            log_level="ERROR",
            max_iter=30,
            maxdiv=5,
            tol=0.001,
            closureConc=[0, 1, 2],
            closureMethod="constantSum",
            closureTarget="default",  # closure constraints
            nonnegConc="all",
            solverConc="nnls",  # Non-negativity constraint
            nonnegSpec="all",
            solverSpec="nnls",  # Non-negativity constraint
        )
        mcr_4.hardConc = [0, 1, 2]
        mcr_4.getConc = kin.fit_to_concentrations
        mcr_4.argsGetConc = ([0, 1, 2], [0, 1, 2], param_to_optimize)
        mcr_4.kwargsGetConc = {
            "ivp_solver_kwargs": {"return_NDDataset": False, "method": "BDF"},
            "optimizer_kwargs": {"options": {"disp": False}},
        }
        mcr_4.fit(XANES, Ckin)
        std_st_Mn = np.std(np.asarray(mcr_4.St.data - ref_XANES[0].data), axis=1)  # noqa: N806
        return [k, m, std_st_Mn[0], std_st_Mn[1], std_st_Mn[2]], mcr_4
    except Exception as e:
        print(f"[Warning] k={k:.3f}, m={m:.3f} failed: {e}")
        return [
            None,
        ], None


for k in tqdm(np.linspace(0.0, 0.5, 100, endpoint=True)):
    for m in np.linspace(0.0, 0.5, 100, endpoint=True):
        result_row = evaluate_kinetics_2(k, m, XANES, ref_XANES)
        if result_row[0]:
            results.append(result_row[0])

result = pd.DataFrame(results, columns=["k", "m", "std_comp1", "std_comp2", "std_comp3"])

# 打印最小标准差对应的参数组合
for i in range(2, 5):
    print(result.iloc[result.iloc[:, i].idxmin(), :])  # type: ignore

# 保存结果
result.to_csv(
    Path.joinpath(path_out, r"P2a_Mn_MCRHM_2reaction_comp_3_A_B_B_AC.csv"),
    sep=",",
    index=False,
    header=True,
)

In [ ]:
# 将上述结果作为初始值，进行优化处理
# kinetic model
ka, kb = 0.075758, 0.000000  # 0.075758, 0.363636
reactions = ("A -> B", "B -> A + C")  # note the coma: this is now a tuple of size 1
species_concentrations = {"A": 0.547, "B": 0, "C": 0.453}

mcr_4 = evaluate_kinetics_2(ka, kb, XANES, ref_XANES)[1]  # 直接使用上面计算的结果
_ = mcr_4.C.T.plot(ylabel=r"relative ratio")
_ = mcr_4.C_constrained.T.plot(clear=False)
_ = mcr_4.St.plot(clear=True, ylims=(-0.1, 1.8))
_ = ref_XANES.plot(clear=False, ls="--", title=r"HM_MCR-ALS and refs")
_ = mcr_4.plotmerit(nb_traces=5)
mcr_4_residual = XANES - mcr_4.inverse_transform()

Mn_MCRHM_TworeactionData = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_4.C.data),
        "MCR_HM_C_constrained": (["spectrum_number", "mcr_number"], mcr_4.C_constrained.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_4.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_4.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_4_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_4.St.shape[0]),
    },
    attrs={
        "reactions": reactions,
        "species_concentrations": str(species_concentrations),
        "k0": str(ka) + ", " + str(kb),
    },
)

(
    xr.merge([Mn_NormalData, Mn_PCA, Mn_MCRHM_TworeactionData], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(
        Path.joinpath(path_out, r"P2a_Mn_MCRHM_2reaction_comp_3_A_B_B_AC.NETCDF4"), mode="w", engine="h5netcdf"
    )
)

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[3]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCRHM_TworeactionData["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"#3"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCRHM_TworeactionData[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCRHM_TworeactionData[r"spectrum_number"],
        Mn_MCRHM_TworeactionData[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.1, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Mn_MCRHM_TworeactionData[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCRHM_TworeactionData["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCRHM_TworeactionData["MCR_HM_St"]["pca_energy"],
        Mn_MCRHM_TworeactionData["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCRHM_TworeactionData["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCRHM_TworeactionData["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCRHM_TworeactionData["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCRHM_TworeactionData["MCR_HM_residual"]["pca_energy"],
        Mn_MCRHM_TworeactionData["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCRHM_TworeactionData_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2a_Mn_MCRHM_TworeactionData_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### Zn, 充电

In [ ]:
# Zn
data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Zn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2a_Zn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Zn = pd.concat([data1, data2.iloc[:, 3:]], axis=1, ignore_index=True)  # noqa: N816
charge_index = [0, 1, 2, *list(range(11, xas_Zn.shape[1], 1))]
xas_Zn = xas_Zn.iloc[:, charge_index]  # noqa: N816

pdf_Zn = xas_Zn.iloc[:, 0:3]  # noqa: N816
pdf_Zn.columns = [r"Energy", r"0.5M_ZnSO4(aq.)", r"ZHS"]
opData_Zn = xas_Zn.iloc[:, [0, *list(range(3, xas_Zn.shape[1]))]]  # noqa: N816
opData_Zn.columns = list(range(0, opData_Zn.shape[1], 1))

In [ ]:
Zn_NormalData = xr.Dataset(
    data_vars={
        "Spectrum": (["energy", "spectrum_number"], opData_Zn.iloc[:, 1:]),
        "Reference": (["energy", "reference_number"], pdf_Zn.iloc[:, 1:]),
    },
    coords={
        "spectrum_number": list(range(0, opData_Zn.shape[1] - 1, 1)),
        "energy": opData_Zn.iloc[:, 0],
        "reference_number": [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"],
    },
)

In [ ]:
data = scp.NDDataset(
    data=Zn_NormalData["Spectrum"].data.T,
    name="opXAS_Mn_CLAESS_2022",
    author="Cheng Liu, Ashely, and Dino Tonti",
    description="opXAS_Mn_CLAESS_2022",
    history="creation",
    title="absorption",
    units=ur.absorbance,
)

data.y = scp.Coord.arange(Zn_NormalData["Spectrum"].shape[1], title="sperctum number")
data.y.labels = Zn_NormalData.coords["spectrum_number"]
data.x = scp.Coord(Zn_NormalData.coords["energy"], title="energy", units=ur.eV)


ref = scp.NDDataset(data=Zn_NormalData["Reference"].data.T, name="ref_NOTOs_2024", units=ur.absorbance)
ref.y = scp.Coord.arange(Zn_NormalData["Reference"].shape[1], title="sperctum number")
ref.x = scp.Coord(Zn_NormalData.coords["energy"], title="energy", units=ur.eV)

_ = data[:, 9643.0:9743.0].plot(title="Normalized Data")
_ = ref[:, 9643.0:9743.0].plot(title=r"Normalized Data and Refs", ls="--", clear=False)

XANES = data[:, 9643.0:9743.0]
ref_XANES = ref[:, 9643.0:9743.0]  # noqa: N816


##### PCA

In [ ]:
# pca analysis
pca = scp.PCA()
pca.fit(XANES)
pca.printev()
# _ = pca.screeplot()  # noqa: ERA001
# loadings = pca.loadings  # noqa: ERA001
# scores = pca.scores  # noqa: ERA001
# _ = pca.loadings.plot(); _.set_ylabel('loadings')  # noqa: ERA001
# _ = pca.scores.T.plot(); _.set_ylabel('scores')  # noqa: ERA001
# _ =pca.scoreplot(pca.scores, 1, 2, color_mapping="labels")  # noqa: ERA001


In [ ]:
Zn_PCA = xr.Dataset(
    data_vars={
        "Explained_variance_ratio": (["spectrum_number"], pca.ev_ratio),
        "Cumulative_explained_variance": (["spectrum_number"], pca.ev_cum),
        "PCA_recon_Data": (["spectrum_number", "pca_energy"], pca.inverse_transform().data),
        "PCA_residual": (["spectrum_number", "pca_energy"], XANES.data - pca.inverse_transform().data),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
    },
)

##### Components = 2

##### 方案一，两个标样做约束

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]
    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr = scp.MCRALS(
    max_iter=100,
    maxdiv=30,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr.hardSpec = [0, 1]
mcr.getSpec = get_St
mcr.hardSt_to_St_idx = [0, 1]
mcr.fit(XANES, St0)
mcr.kwargsGetConc = {"ivp_solver_kwargs": {"return_NDDataset": False}, "optimizer_kwargs": {"options": {"disp": False}}}

_ = mcr.C.T.plot(clear=True, title=r"Concentration")
_ = mcr.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--")

_ = mcr.plotmerit(nb_traces=5, clear=True)
mcr_residual = XANES - mcr.inverse_transform()

In [ ]:
Zn_MCR_2 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr.St.shape[0]),
    },
)

(xr.merge([Zn_NormalData, Zn_PCA, Zn_MCR_2], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2a_Zn_MCR_2.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(0.0, 1.0, Zn_MCR_2["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Zn_MCR_2[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Zn_MCR_2[r"spectrum_number"],
        Zn_MCR_2[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[1][i],
        markeredgecolor=xascolors[1][i],
        markersize=8,
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.5, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Zn_MCR_2[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_NormalData["Reference"].shape[1]):
    ax.plot(
        Zn_NormalData["Reference"]["energy"],
        Zn_NormalData["Reference"][:, i],
        c=xascolors[1][i],
        ls="--",
        lw=1.0,
        label=labels[1][i],
        zorder=5,
    )
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(Zn_MCR_2["MCR_HM_St"].shape[0]):
    ax.plot(
        Zn_MCR_2["MCR_HM_St"]["pca_energy"],
        Zn_MCR_2["MCR_HM_St"][i, :],
        c=xascolors[1][i],
        label=None,
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_MCR_2["MCR_HM_residual"].shape[0]):
    ax.plot(
        Zn_MCR_2["MCR_HM_recon_Data"]["pca_energy"],
        Zn_MCR_2["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Zn_MCR_2["MCR_HM_residual"]["pca_energy"],
        Zn_MCR_2["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(-0.4, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2a_Zn_MCR_2_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2a_Zn_MCR_2_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### 方案二， Hard-Model ( A --> B ), ZHS 0.688, 和 0.5M Zn2+ 0.312

In [ ]:
# kinetic model
from tqdm.notebook import tqdm

reactions = ("A -> B",)  # note the coma: this is now a tuple of size 1, 不管方程式的系数：这个从 Zn 那边验证过
species_concentrations = {"A": 0.688, "B": 0.312}
results = []


def evaluate_kinetics(k, XANES, ref_XANES):  # noqa: N803
    try:
        k0 = np.array([
            k,
        ])  # as we have a single reaction, we must use a single rate constant.
        kin = scp.ActionMassKinetics(reactions, species_concentrations, k0)
        Ckin = kin.integrate(XANES.y.data)  # noqa: N806

        param_to_optimize = {
            "k[0]": k,
        }
        mcr_2 = scp.MCRALS(
            max_iter=100,
            maxdiv=30,
            log_level="ERROR",
            tol=0.001,
            closureConc=[0, 1],
            closureMethod="constantSum",
            closureTarget="default",  # closure constraints
            nonnegConc="all",
            solverConc="nnls",  # Non-negativity constraint
            nonnegSpec="all",
            solverSpec="nnls",  # Non-negativity constraint
        )
        mcr_2.hardConc = [0, 1]
        mcr_2.getConc = kin.fit_to_concentrations
        mcr_2.argsGetConc = ([0, 1], [0, 1], param_to_optimize)
        mcr_2.kwargsGetConc = {
            "ivp_solver_kwargs": {"return_NDDataset": False},
            "optimizer_kwargs": {"options": {"disp": False}},
        }
        mcr_2.fit(XANES, Ckin)
        std_st_Zn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1)  # Zn2+  # noqa: N806
        return [k, std_st_Zn[0], std_st_Zn[1]], mcr_2
        # std_st_Zn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1) # Zn2+ 作为参考  # noqa: ERA001
        # std_st_Zn2 = np.std(np.asarray(mcr_2.St.data - ref_XANES[1].data), axis=1) # ZHS 作为参考  # noqa: ERA001
        # std_Mn= [std_st_Zn[i] + std_st_Zn2[j] for i in range(len(std_st_Zn)) for j in range(len(std_st_Zn2)) if i != j]  # noqa: E501, ERA001
        # return [k, std_st_Zn[0], std_st_Zn[1]], mcr_2  # noqa: ERA001
    except Exception as e:
        print(f"[Warning] k={k:.3f}, failed: {e}")
        return [
            None,
        ], None


for k in tqdm(np.linspace(0.0, 0.5, 500, endpoint=True)):
    result_row = evaluate_kinetics(k, XANES, ref_XANES)
    if result_row[0]:
        results.append(result_row[0])

result = pd.DataFrame(results, columns=["k", "std_comp1", "std_comp2"])

# 打印最小标准差对应的参数组合
for i in range(1, 3):
    print(result.iloc[result.iloc[:, i].idxmin(), :])  # type: ignore

# 保存结果
result.to_csv(
    Path.joinpath(path_out, r"P2a_Zn_MCRHM_1reaction_comp_2_A_B.csv"),
    sep=",",
    index=False,
    header=True,
)

In [ ]:
# 将上述结果作为初始值，进行最后的结果
#  kinetic model
ka = 0.126253
reactions = ("A -> B",)  # note the coma: this is now a tuple of size 1
species_concentrations = {"A": 0.688, "B": 0.312}

mcr_2 = evaluate_kinetics(ka, XANES, ref_XANES)[1]  # 直接使用上面计算的结果
_ = mcr_2.C.T.plot(ylabel=r"relative ratio")
_ = mcr_2.C_constrained.T.plot(clear=False)
_ = mcr_2.St.plot(clear=True, ylims=(-0.1, 1.8))
_ = ref_XANES.plot(clear=False, ls="--")

_ = mcr_2.plotmerit(nb_traces=5)
mcr_2_residual = XANES - mcr_2.inverse_transform()

Zn_MCRHM_OnereactionData = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_2.C.data),
        "MCR_HM_C_constrained": (["spectrum_number", "mcr_number"], mcr_2.C_constrained.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_2.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_2.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_2_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_2.St.shape[0]),
    },
    attrs={
        "reactions": reactions,
        "species_concentrations": str(species_concentrations),
        "k0": ka,
    },
)

(
    xr.merge([Zn_NormalData, Zn_PCA, Zn_MCRHM_OnereactionData], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(
        Path.joinpath(path_out, r"P2a_Zn_MCRHM_1reaction_comp_2_A_B_2.NETCDF4"), mode="w", engine="h5netcdf"
    )
)

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 3.3))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4]], [colors[1], colors[2]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(0.0, 1.0, Zn_MCRHM_OnereactionData["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Zn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Zn_MCRHM_OnereactionData[r"spectrum_number"],
        Zn_MCRHM_OnereactionData[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[1][i],
        markeredgecolor=xascolors[1][i],
        markersize=8,
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.3, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

line = []
for i in range(Zn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"].shape[1]):
    (lineB,) = ax.plot(  # noqa: N816
        Zn_MCRHM_OnereactionData[r"spectrum_number"],
        Zn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"][:, i],
        c="b",
        label=labels[1][i],
        ls="--",
    )
    line.append(lineB)
ax.legend(
    handles=[line[0]],
    labels=[r"Initial Concentration"],
    loc="upper left",
    bbox_to_anchor=(0.1, 0.15),
    ncols=1,
    handlelength=3,
    labelcolor="linecolor",
    fontsize=9,
)


ax.set_xlim(-0.5, Zn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Zn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_NormalData["Reference"].shape[1]):
    ax.plot(
        Zn_NormalData["Reference"]["energy"],
        Zn_NormalData["Reference"][:, i],
        c=xascolors[1][i],
        ls="--",
        lw=1.0,
        label=labels[1][i],
        zorder=5,
    )
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(Zn_MCRHM_OnereactionData["MCR_HM_St"].shape[0]):
    ax.plot(
        Zn_MCRHM_OnereactionData["MCR_HM_St"]["pca_energy"],
        Zn_MCRHM_OnereactionData["MCR_HM_St"][i, :],
        c=xascolors[1][i],
        label=None,
        ls="-",
        lw=1.0,
    )

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_MCRHM_OnereactionData["MCR_HM_residual"].shape[0]):
    ax.plot(
        Zn_MCRHM_OnereactionData["MCR_HM_recon_Data"]["pca_energy"],
        Zn_MCRHM_OnereactionData["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Zn_MCRHM_OnereactionData["MCR_HM_residual"]["pca_energy"],
        Zn_MCRHM_OnereactionData["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(-0.4, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2a_Zn_MCRHM_1reaction_comp_2_A_B_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2a_Zn_MCRHM_1reaction_comp_2_A_B_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### Components = 3

##### 方案一，两个标样做约束

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]
    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
        XANES[-1].squeeze().data,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr_3 = scp.MCRALS(
    max_iter=300,
    maxdiv=100,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1, 2],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr_3.hardSpec = [0, 1, 2]
mcr_3.getSpec = get_St
mcr_3.hardSt_to_St_idx = [0, 1, 2]
mcr_3.fit(XANES, St0)
mcr_3.kwargsGetConc = {
    "ivp_solver_kwargs": {"return_NDDataset": False},
    "optimizer_kwargs": {"options": {"disp": False}},
}

_ = mcr_3.C.T.plot(clear=True, title=r"Concentration")
_ = mcr_3.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--")

_ = mcr_3.plotmerit(nb_traces=5, clear=True)
mcr_3_residual = XANES - mcr_3.inverse_transform()

Zn_MCR_3 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_3.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_3.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_3.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_3_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_3.St.shape[0]),
    },
)

(xr.merge([Zn_NormalData, Zn_PCA, Zn_MCR_3], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2a_Zn_MCR_3.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[3]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(0.0, 1.0, Zn_MCR_3["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"#3"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Zn_MCR_3[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Zn_MCR_3[r"spectrum_number"],
        Zn_MCR_3[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[1][i],
        markeredgecolor=xascolors[1][i],
        markersize=8,
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.0, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Zn_MCR_3[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(-0.05, 1.1)
ax.set_ylabel(r"Zn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)


for i in range(Zn_MCR_3["MCR_HM_St"].shape[0]):
    ax.plot(
        Zn_MCR_3["MCR_HM_St"]["pca_energy"],
        Zn_MCR_3["MCR_HM_St"][i, :],
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Zn_NormalData["Reference"].shape[1]):
    ax.plot(
        Zn_NormalData["Reference"]["energy"],
        Zn_NormalData["Reference"][:, i],
        c=xascolors[1][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_MCR_3["MCR_HM_residual"].shape[0]):
    ax.plot(
        Zn_MCR_3["MCR_HM_recon_Data"]["pca_energy"],
        Zn_MCR_3["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Zn_MCR_3["MCR_HM_residual"]["pca_energy"],
        Zn_MCR_3["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(-0.4, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2a_Zn_MCR_3_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2a_Zn_MCR_3_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()


##### 方案二，Hard-Model ( "A -> B", "B -> C" ), 利用 CLAESS Refers 确定初始浓度值，-20 到 80 eV, 1.0 和 0.0

In [ ]:
from tqdm.notebook import tqdm

reactions = ("A -> B", "B -> C")
species_concentrations = {"A": 0.688, "B": 0.312, "C": 0.0}
results = []


def evaluate_kinetics_2(k, m, XANES, ref_XANES):  # noqa: N803
    try:
        k0 = np.array([k, m])
        kin = scp.ActionMassKinetics(reactions, species_concentrations, k0)
        Ckin = kin.integrate(XANES.y.data)  # noqa: N806

        param_to_optimize = {"k[0]": k, "k[1]": m}
        mcr_4 = scp.MCRALS(
            log_level="ERROR",
            max_iter=30,
            maxdiv=5,
            tol=0.001,
            closureConc=[0, 1, 2],
            closureMethod="constantSum",
            closureTarget="default",  # closure constraints
            nonnegConc="all",
            solverConc="nnls",  # Non-negativity constraint
            nonnegSpec="all",
            solverSpec="nnls",  # Non-negativity constraint
        )
        mcr_4.hardConc = [0, 1, 2]
        mcr_4.getConc = kin.fit_to_concentrations
        mcr_4.argsGetConc = ([0, 1, 2], [0, 1, 2], param_to_optimize)
        mcr_4.kwargsGetConc = {
            "ivp_solver_kwargs": {"return_NDDataset": False, "method": "BDF"},
            "optimizer_kwargs": {"options": {"disp": False}},
        }
        mcr_4.fit(XANES, Ckin)
        std_st_Mn = np.std(np.asarray(mcr_4.St.data - ref_XANES[0].data), axis=1)  # noqa: N806
        return [k, m, std_st_Mn[0], std_st_Mn[1], std_st_Mn[2]], mcr_4
    except Exception as e:
        print(f"[Warning] k={k:.3f}, m={m:.3f} failed: {e}")
        return [
            None,
        ], None


for k in tqdm(np.linspace(0.0, 0.5, 100, endpoint=True)):
    for m in np.linspace(0.0, 0.5, 100, endpoint=True):
        result_row = evaluate_kinetics_2(k, m, XANES, ref_XANES)
        if result_row[0]:
            results.append(result_row[0])

result = pd.DataFrame(results, columns=["k", "m", "std_comp1", "std_comp2", "std_comp3"])

# 打印最小标准差对应的参数组合
for i in range(2, 5):
    print(result.iloc[result.iloc[:, i].idxmin(), :])  # type: ignore

# 保存结果
result.to_csv(
    Path.joinpath(path_out, r"P2a_Zn_MCRHM_2reaction_comp_3_A_B_B_C.csv"),
    sep=",",
    index=False,
    header=True,
)

In [ ]:
# 将上述结果作为初始值，进行优化处理
# kinetic model
ka, kb = 0.459596, 0.212121
reactions = ("A -> B", "B -> C")
species_concentrations = {"A": 1.0, "B": 0.0, "C": 0.0}

mcr_4 = evaluate_kinetics_2(ka, kb, XANES, ref_XANES)[1]  # 直接使用上面计算的结果
_ = mcr_4.C.T.plot(ylabel=r"relative ratio")
_ = mcr_4.C_constrained.T.plot(clear=False)
_ = mcr_4.St.plot(clear=True, ylims=(-0.1, 1.8))
_ = ref_XANES.plot(clear=False, ls="--")
_ = mcr_4.plotmerit(nb_traces=5)
mcr_4_residual = XANES - mcr_4.inverse_transform()

Zn_MCRHM_TworeactionData = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_4.C.data),
        "MCR_HM_C_constrained": (["spectrum_number", "mcr_number"], mcr_4.C_constrained.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_4.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_4.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_4_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_4.St.shape[0]),
    },
    attrs={
        "reactions": reactions,
        "species_concentrations": str(species_concentrations),
        "k0": str(ka) + ", " + str(kb),
    },
)

(
    xr.merge([Zn_NormalData, Zn_PCA, Zn_MCRHM_TworeactionData], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(
        Path.joinpath(path_out, r"P2a_Zn_MCRHM_2reaction_comp_3_A_B_B_C.NETCDF4"), mode="w", engine="h5netcdf"
    )
)

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[3]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(0.0, 1.0, Zn_MCRHM_TworeactionData["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"#3"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Zn_MCRHM_TworeactionData[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Zn_MCRHM_TworeactionData[r"spectrum_number"],
        Zn_MCRHM_TworeactionData[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[1][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.1, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Zn_MCRHM_TworeactionData[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_MCRHM_TworeactionData["MCR_HM_St"].shape[0]):
    ax.plot(
        Zn_MCRHM_TworeactionData["MCR_HM_St"]["pca_energy"],
        Zn_MCRHM_TworeactionData["MCR_HM_St"][i, :],
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Zn_NormalData["Reference"].shape[1]):
    ax.plot(
        Zn_NormalData["Reference"]["energy"],
        Zn_NormalData["Reference"][:, i],
        c=xascolors[1][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_MCRHM_TworeactionData["MCR_HM_residual"].shape[0]):
    ax.plot(
        Zn_MCRHM_TworeactionData["MCR_HM_recon_Data"]["pca_energy"],
        Zn_MCRHM_TworeactionData["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Zn_MCRHM_TworeactionData["MCR_HM_residual"]["pca_energy"],
        Zn_MCRHM_TworeactionData["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(-0.4, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2a_Zn_MCRHM_TworeactionData_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2a_Zn_MCRHM_TworeactionData_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

#### P2b MCR-ALS

##### Mn，充电

In [ ]:
# 读取 reference + operando 数据
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2"  # noqa: RUF001
)

data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Mn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Mn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Mn = pd.concat([data1, data2.iloc[:, 5:]], axis=1, ignore_index=True)  # noqa: N816
charge_index = [0, 1, 2, 3, 4, *list(range(14, xas_Mn.shape[1], 1))]
xas_Mn = xas_Mn.iloc[:, charge_index]  # noqa: N816
pdf_Mn = xas_Mn.iloc[:, 0:5]  # noqa: N816
pdf_Mn.columns = [
    r"Energy",
    r"0.2M_MnSO4(aq.)",
    r"alpha_MnO2_Electrode",
    r"alpha_MnO2_Powder",
    r"FC1st",
]
opData_Mn = xas_Mn.iloc[:, [0, *list(range(5, xas_Mn.shape[1]))]]  # noqa: N816
opData_Mn.columns = list(range(0, opData_Mn.shape[1], 1))

In [ ]:
Mn_NormalData = xr.Dataset(
    data_vars={
        "Spectrum": (["energy", "spectrum_number"], opData_Mn.iloc[:, 1:]),
        "Reference": (["energy", "reference_number"], pdf_Mn.iloc[:, [1, 2, 4]]),
    },
    coords={
        "spectrum_number": list(range(0, opData_Mn.shape[1] - 1, 1)),
        "energy": opData_Mn.iloc[:, 0],
        "reference_number": [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1st"],
    },
)

In [ ]:
data = scp.NDDataset(
    data=Mn_NormalData["Spectrum"].data.T,
    name="opXAS_Mn_CLAESS_2022",
    author="Cheng Liu, Ashely, and Dino Tonti",
    description="opXAS_Mn_CLAESS_2022",
    history="creation",
    title="absorption",
    units=ur.absorbance,
)

data.y = scp.Coord.arange(Mn_NormalData["Spectrum"].shape[1], title="sperctum number")
data.y.labels = Mn_NormalData.coords["spectrum_number"]
data.x = scp.Coord(Mn_NormalData.coords["energy"], title="energy", units=ur.eV)


ref = scp.NDDataset(data=Mn_NormalData["Reference"].data.T, name="ref_NOTOs_2024", units=ur.absorbance)
ref.y = scp.Coord.arange(Mn_NormalData["Reference"].shape[1], title="sperctum number")
ref.x = scp.Coord(Mn_NormalData.coords["energy"], title="energy", units=ur.eV)

_ = data[:, 6530.0:6630.0].plot(title="Normalized Data")
_ = ref[:, 6530.0:6630.0].plot(title=r"Normalized Data and Refs", ls="--", clear=False)

XANES = data[:, 6530.0:6630.0]
ref_XANES = ref[:, 6530.0:6630.0]  # noqa: N816

##### PCA

In [ ]:
# # pca analysis
pca = scp.PCA()
pca.fit(XANES)
pca.printev()
# _ = pca.screeplot()  # noqa: ERA001
# loadings = pca.loadings  # noqa: ERA001
# scores = pca.scores  # noqa: ERA001
# _ = pca.loadings.plot(); _.set_ylabel('loadings')  # noqa: ERA001
# _ = pca.scores.T.plot(); _.set_ylabel('scores')  # noqa: ERA001
# _ =pca.scoreplot(pca.scores, 1, 2, color_mapping="labels")  # noqa: ERA001

In [ ]:
Mn_PCA = xr.Dataset(
    data_vars={
        "Explained_variance_ratio": (["spectrum_number"], pca.ev_ratio),
        "Cumulative_explained_variance": (["spectrum_number"], pca.ev_cum),
        "PCA_recon_Data": (["spectrum_number", "pca_energy"], pca.inverse_transform().data),
        "PCA_residual": (["spectrum_number", "pca_energy"], XANES.data - pca.inverse_transform().data),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
    },
)

##### Components = 2

##### 方案一，两个标样做约束

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]
    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr = scp.MCRALS(
    max_iter=300,
    maxdiv=100,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr.hardSpec = [0, 1]
mcr.getSpec = get_St
mcr.hardSt_to_St_idx = [0, 1]
mcr.fit(XANES, St0)
mcr.kwargsGetConc = {"ivp_solver_kwargs": {"return_NDDataset": False}, "optimizer_kwargs": {"options": {"disp": False}}}

_ = mcr.C.T.plot(clear=True, title=r"Concentration")
_ = mcr.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--", title=r"MCR-ALS and Refs")

_ = mcr.plotmerit(nb_traces=5, clear=True)
mcr_residual = XANES - mcr.inverse_transform()

In [ ]:
Mn_MCR_2 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr.St.shape[0]),
    },
)

(xr.merge([Mn_NormalData, Mn_PCA, Mn_MCR_2], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2b_Mn_MCR_2.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCR_2["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1st"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCR_2[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCR_2[r"spectrum_number"],
        Mn_MCR_2[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.4, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Mn_MCR_2[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.1, 0.9)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=labels[0][i],
        zorder=5,
    )
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(Mn_MCR_2["MCR_HM_St"].shape[0]):
    ax.plot(Mn_MCR_2["MCR_HM_St"]["pca_energy"], Mn_MCR_2["MCR_HM_St"][i, :], c=xascolors[0][i], label=None, ls="-", lw=1.0)

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCR_2["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCR_2["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCR_2["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCR_2["MCR_HM_residual"]["pca_energy"],
        Mn_MCR_2["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCR_2_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCR_2_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### 方案二，Hard-Model (A --> B), 利用 LCF, Mn2+, 0.633? 和 aMnO2, 0.367 或者 Mn2+, 0.515 和 FC1st, 0.485

In [ ]:
# kinetic model
# A，用的是 Mn2+ + PRISTINE
from tqdm.notebook import tqdm

reactions = ("A -> B",)  # note the coma: this is now a tuple of size 1, 不管方程式的系数：这个从 Zn 那边验证过
species_concentrations = {"A": 0.633, "B": 0.367}  # A = Mn2+, B = MnO2
results = []


def evaluate_kinetics(k, XANES, ref_XANES):  # noqa: N803
    try:
        k0 = np.array([
            k,
        ])  # as we have a single reaction, we must use a single rate constant.
        kin = scp.ActionMassKinetics(reactions, species_concentrations, k0)
        Ckin = kin.integrate(XANES.y.data)  # noqa: N806

        param_to_optimize = {
            "k[0]": k,
        }
        mcr_2 = scp.MCRALS(
            max_iter=100,
            maxdiv=10,
            log_level="ERROR",
            tol=0.001,
            closureConc=[0, 1],
            closureMethod="constantSum",
            closureTarget="default",  # closure constraints
            nonnegConc="all",
            solverConc="nnls",  # Non-negativity constraint
            nonnegSpec="all",
            solverSpec="nnls",  # Non-negativity constraint
        )
        mcr_2.hardConc = [0, 1]
        mcr_2.getConc = kin.fit_to_concentrations
        mcr_2.argsGetConc = ([0, 1], [0, 1], param_to_optimize)
        mcr_2.kwargsGetConc = {
            "ivp_solver_kwargs": {"return_NDDataset": False},
            "optimizer_kwargs": {"options": {"disp": False}},
        }
        mcr_2.fit(XANES, Ckin)
        # std_st_Mn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1) # Mn2+  # noqa: ERA001
        # return [k, *std_st_Mn], mcr_2  # noqa: ERA001
        std_st_Mn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1)  # Mn2+  # noqa: N806
        std_st_Mn2 = np.std(np.asarray(mcr_2.St.data - ref_XANES[1].data), axis=1)  # MnO2  # noqa: N806
        std_Mn = [std_st_Mn[i] + std_st_Mn2[j] for i in range(len(std_st_Mn)) for j in range(len(std_st_Mn2)) if i != j]  # noqa: N806
        return [k, std_Mn[0], std_Mn[1]], mcr_2
    except Exception as e:
        print(f"[Warning] k={k:.3f}, failed: {e}")
        return None


for k in tqdm(np.linspace(0.0, 0.5, 500, endpoint=True)):
    result_row = evaluate_kinetics(k, XANES, ref_XANES)
    if result_row[0]:  # type: ignore
        results.append(result_row[0])  # type: ignore

result = pd.DataFrame(results, columns=["k", "std_comp1", "std_comp2"])

# 打印最小标准差对应的参数组合
for i in range(1, 3):
    print(result.iloc[result.iloc[:, i].idxmin(), :])  # type: ignore

# 保存结果
result.to_csv(
    Path.joinpath(path_out, r"P2b_Mn_MCRHM_1reaction_comp_2_A_B.csv"),
    sep=",",
    index=False,
    header=True,
)

In [ ]:
# 将上述结果作为初始值，进行最后的结果
#  kinetic model
ka = 0.017034
reactions = ("A -> B",)  # note the coma: this is now a tuple of size 1
species_concentrations = {"A": 0.633, "B": 0.367}  # A = Mn2+, B = MnO2

mcr_2 = evaluate_kinetics(ka, XANES, ref_XANES)[1]  # type: ignore
_ = mcr_2.C.T.plot(ylabel=r"relative ratio")
_ = mcr_2.C_constrained.T.plot(clear=False)
_ = mcr_2.St.plot(clear=True, ylims=(-0.1, 1.8))
_ = ref_XANES.plot(clear=False, ls="--")

_ = mcr_2.plotmerit(nb_traces=5)
mcr_2_residual = XANES - mcr_2.inverse_transform()

Mn_MCRHM_OnereactionData = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_2.C.data),
        "MCR_HM_C_constrained": (["spectrum_number", "mcr_number"], mcr_2.C_constrained.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_2.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_2.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_2_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_2.St.shape[0]),
    },
    attrs={"reactions": reactions, "species_concentrations": str(species_concentrations), "k0": ka},
)

(
    xr.merge([Mn_NormalData, Mn_PCA, Mn_MCRHM_OnereactionData], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(
        Path.joinpath(path_out, r"P2b_Mn_MCRHM_1reaction_comp_2_A_B.NETCDF4"), mode="w", engine="h5netcdf"
    )
)

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCRHM_TworeactionData["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1st"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCRHM_OnereactionData[r"spectrum_number"],
        Mn_MCRHM_OnereactionData[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.3, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

line = []
for i in range(Mn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"].shape[1]):
    (lineB,) = ax.plot(  # noqa: N816
        Mn_MCRHM_OnereactionData[r"spectrum_number"],
        Mn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"][:, i],
        c="b",
        label=labels[0][i],
        ls="--",
    )
    line.append(lineB)
ax.legend(
    handles=[line[0]],
    labels=[r"Initial Concentration"],
    loc="upper left",
    bbox_to_anchor=(0.3, 0.15),
    ncols=1,
    handlelength=3,
    labelcolor="linecolor",
    fontsize=9,
)

ax.set_xlim(-0.5, Mn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=labels[0][i],
        zorder=5,
    )
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(Mn_MCRHM_OnereactionData["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCRHM_OnereactionData["MCR_HM_St"]["pca_energy"],
        Mn_MCRHM_OnereactionData["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=None,
        ls="-",
        lw=1.0,
    )

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCRHM_TworeactionData["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCRHM_TworeactionData["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCRHM_TworeactionData["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCRHM_TworeactionData["MCR_HM_residual"]["pca_energy"],
        Mn_MCRHM_TworeactionData["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCRHM_1reaction_comp_2_A_B_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCRHM_1reaction_comp_2_A_B_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

In [ ]:
# kinetic model
# B，用的是 Mn2+ + FC1st
from tqdm.notebook import tqdm

reactions = ("A -> B",)  # note the coma: this is now a tuple of size 1, 不管方程式的系数：这个从 Zn 那边验证过
species_concentrations = {"A": 0.515, "B": 0.485}  # A = Mn2+, B = FC1st
results = []


def evaluate_kinetics(k, XANES, ref_XANES):  # noqa: N803
    try:
        k0 = np.array([
            k,
        ])  # as we have a single reaction, we must use a single rate constant.
        kin = scp.ActionMassKinetics(reactions, species_concentrations, k0)
        Ckin = kin.integrate(XANES.y.data)  # noqa: N806

        param_to_optimize = {
            "k[0]": k,
        }
        mcr_2 = scp.MCRALS(
            max_iter=100,
            maxdiv=10,
            log_level="ERROR",
            tol=0.001,
            closureConc=[0, 1],
            closureMethod="constantSum",
            closureTarget="default",  # closure constraints
            nonnegConc="all",
            solverConc="nnls",  # Non-negativity constraint
            nonnegSpec="all",
            solverSpec="nnls",  # Non-negativity constraint
        )
        mcr_2.hardConc = [0, 1]
        mcr_2.getConc = kin.fit_to_concentrations
        mcr_2.argsGetConc = ([0, 1], [0, 1], param_to_optimize)
        mcr_2.kwargsGetConc = {
            "ivp_solver_kwargs": {"return_NDDataset": False},
            "optimizer_kwargs": {"options": {"disp": False}},
        }
        mcr_2.fit(XANES, Ckin)
        # std_st_Mn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1) # Mn2+  # noqa: ERA001
        # return [k, std_st_Mn[0], std_st_Mn[1]], mcr_2  # noqa: ERA001
        std_st_Mn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1)  # Mn2+  # noqa: N806
        std_st_Mn2 = np.std(np.asarray(mcr_2.St.data - ref_XANES[2].data), axis=1)  # MnO2  # noqa: N806
        std_Mn = [std_st_Mn[i] + std_st_Mn2[j] for i in range(len(std_st_Mn)) for j in range(len(std_st_Mn2)) if i != j]  # noqa: N806
        return [k, std_Mn[0], std_Mn[1]], mcr_2
    except Exception as e:
        print(f"[Warning] k={k:.3f}, failed: {e}")
        return None


for k in tqdm(np.linspace(0.0, 0.5, 500, endpoint=True)):
    result_row = evaluate_kinetics(k, XANES, ref_XANES)
    if result_row[0]:  # type: ignore
        results.append(result_row[0])  # type: ignore

result = pd.DataFrame(results, columns=["k", "std_comp1", "std_comp2"])

# 打印最小标准差对应的参数组合
for i in range(1, 3):
    print(result.iloc[result.iloc[:, i].idxmin(), :])  # type: ignore

# 保存结果
result.to_csv(
    Path.joinpath(path_out, r"P2b_Mn_MCRHM_1reaction_comp_2_A_B_FC1st_Mn2+.csv"),
    sep=",",
    index=False,
    header=True,
)

In [ ]:
# 将上述结果作为初始值，进行最后的结果
#  kinetic model
ka = 0.015030
reactions = ("A -> B",)  # note the coma: this is now a tuple of size 1
species_concentrations = {"A": 0.515, "B": 0.485}  # A = Mn2+, B = FC1st

mcr_2 = evaluate_kinetics(ka, XANES, ref_XANES)[1]  # type: ignore
_ = mcr_2.C.T.plot(ylabel=r"relative ratio")
_ = mcr_2.C_constrained.T.plot(clear=False)
_ = mcr_2.St.plot(clear=True, ylims=(-0.1, 1.8))
_ = ref_XANES.plot(clear=False, ls="--")

_ = mcr_2.plotmerit(nb_traces=5)
mcr_2_residual = XANES - mcr_2.inverse_transform()

Mn_MCRHM_OnereactionData = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_2.C.data),
        "MCR_HM_C_constrained": (["spectrum_number", "mcr_number"], mcr_2.C_constrained.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_2.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_2.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_2_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_2.St.shape[0]),
    },
    attrs={"reactions": reactions, "species_concentrations": str(species_concentrations), "k0": ka},
)

(
    xr.merge([Mn_NormalData, Mn_PCA, Mn_MCRHM_OnereactionData], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(
        Path.joinpath(path_out, r"P2b_Mn_MCRHM_1reaction_comp_2_A_B_FC1st_Mn2+.NETCDF4"), mode="w", engine="h5netcdf"
    )
)

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCRHM_OnereactionData["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1st"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCRHM_OnereactionData[r"spectrum_number"],
        Mn_MCRHM_OnereactionData[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.1, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

line = []
for i in range(Mn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"].shape[1]):
    (lineB,) = ax.plot(  # noqa: N816
        Mn_MCRHM_OnereactionData[r"spectrum_number"],
        Mn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"][:, i],
        c="b",
        label=labels[0][i],
        ls="--",
    )
    line.append(lineB)
ax.legend(
    handles=[line[0]],
    labels=[r"Initial Concentration"],
    loc="upper left",
    bbox_to_anchor=(0.3, 0.15),
    ncols=1,
    handlelength=3,
    labelcolor="linecolor",
    fontsize=9,
)

ax.set_xlim(-0.5, Mn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.1, 0.8)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.05))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=labels[0][i],
        zorder=5,
    )
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(Mn_MCRHM_OnereactionData["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCRHM_OnereactionData["MCR_HM_St"]["pca_energy"],
        Mn_MCRHM_OnereactionData["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=None,
        ls="-",
        lw=1.0,
    )

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCRHM_OnereactionData["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCRHM_OnereactionData["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCRHM_OnereactionData["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCRHM_OnereactionData["MCR_HM_residual"]["pca_energy"],
        Mn_MCRHM_OnereactionData["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCRHM_1reaction_comp_2_A_B_FC1st_Mn2+_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCRHM_1reaction_comp_2_A_B_FC1st_Mn2+_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### Components = 3

##### 方案一，三个标样做约束， Mn2+，MnO2，FullCharge

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]
    St[2, :] = ref_XANES[2]
    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
        ref_XANES[2].squeeze().data,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr_3 = scp.MCRALS(
    max_iter=300,
    maxdiv=100,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1, 2],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr_3.hardSpec = [0, 1, 2]
mcr_3.getSpec = get_St
mcr_3.hardSt_to_St_idx = [0, 1, 2]
mcr_3.fit(XANES, St0)
mcr_3.kwargsGetConc = {
    "ivp_solver_kwargs": {"return_NDDataset": False},
    "optimizer_kwargs": {"options": {"disp": False}},
}

_ = mcr_3.C.T.plot(clear=True, title=r"Concentration")
_ = mcr_3.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--", title=r"MCR-ALS and Refs")

_ = mcr_3.plotmerit(nb_traces=5, clear=True)
mcr_3_residual = XANES - mcr_3.inverse_transform()

In [ ]:
Mn_MCR_3 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_3.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_3.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_3.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_3_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_3.St.shape[0]),
    },
)

(xr.merge([Mn_NormalData, Mn_PCA, Mn_MCR_3], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2b_Mn_MCR_3.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCR_3["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1st"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCR_3[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCR_3[r"spectrum_number"],
        Mn_MCR_3[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.45, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Mn_MCR_3[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)


for i in range(Mn_MCR_3["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_St"]["pca_energy"],
        Mn_MCR_3["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCR_3["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCR_3["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCR_3["MCR_HM_residual"]["pca_energy"],
        Mn_MCR_3["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCR_3_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCR_3_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### 方案二，三个标样做约束， Mn2+，MnO2，Dino 从 FullCharge 分离出来的

In [ ]:
path_dino = Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\XAS\Operando\αMnO2\XAS\CLAESS_2022-10\Results\cell2\Dino")  # noqa: E501, RUF001
dino = pd.read_csv(Path.joinpath(path_dino, r"FC1st_3rd_phase.txt"), sep=r"\s+", header=None, index_col=None)
dino = dino[(dino.iloc[:, 0] >= 6530.0) & (dino.iloc[:, 0] <= 6630.0)].copy()

In [ ]:
from scipy.interpolate import interp1d

interp_func = interp1d(dino.iloc[:, 0], dino.iloc[:, 3], kind="quadratic", bounds_error=False, fill_value=0)
dino_interp = interp_func(ref_XANES.x.data)

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]
    St[2, :] = dino_interp
    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
        dino_interp,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr_3 = scp.MCRALS(
    max_iter=300,
    maxdiv=100,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1, 2],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr_3.hardSpec = [0, 1, 2]
mcr_3.getSpec = get_St
mcr_3.hardSt_to_St_idx = [0, 1, 2]
mcr_3.fit(XANES, St0)
mcr_3.kwargsGetConc = {
    "ivp_solver_kwargs": {"return_NDDataset": False},
    "optimizer_kwargs": {"options": {"disp": False}},
}

_ = mcr_3.C.T.plot(clear=True, title=r"Concentration")
_ = mcr_3.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--", title=r"MCR-ALS and Refs")

_ = mcr_3.plotmerit(nb_traces=5, clear=True)
mcr_3_residual = XANES - mcr_3.inverse_transform()

In [ ]:
Mn_MCR_3 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_3.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_3.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_3.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_3_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_3.St.shape[0]),
    },
)

(xr.merge([Mn_NormalData, Mn_PCA, Mn_MCR_3], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2b_Mn_MCR_3_FC.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCR_3["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"FC1stDino"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCR_3[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCR_3[r"spectrum_number"],
        Mn_MCR_3[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.5, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Mn_MCR_3[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)


for i in range(Mn_MCR_3["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_St"]["pca_energy"],
        Mn_MCR_3["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCR_3["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCR_3["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCR_3["MCR_HM_residual"]["pca_energy"],
        Mn_MCR_3["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCR_3_FC_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCR_3_FC_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### 方案三，两个标样做约束， Mn2+，MnO2

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]

    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
        XANES[0].squeeze().data,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr_3 = scp.MCRALS(
    max_iter=300,
    maxdiv=100,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1, 2],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr_3.hardSpec = [0, 1, 2]
mcr_3.getSpec = get_St
mcr_3.hardSt_to_St_idx = [0, 1, 2]
mcr_3.fit(XANES, St0)
mcr_3.kwargsGetConc = {
    "ivp_solver_kwargs": {"return_NDDataset": False},
    "optimizer_kwargs": {"options": {"disp": False}},
}

_ = mcr_3.C.T.plot(clear=True, title=r"Concentration")
_ = mcr_3.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--", title=r"MCR-ALS and Refs")

_ = mcr_3.plotmerit(nb_traces=5, clear=True)
mcr_3_residual = XANES - mcr_3.inverse_transform()

In [ ]:
Mn_MCR_3 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_3.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_3.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_3.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_3_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_3.St.shape[0]),
    },
)

(xr.merge([Mn_NormalData, Mn_PCA, Mn_MCR_3], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2b_Mn_MCR_3_2.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[6]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCR_3["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"#3"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCR_3[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCR_3[r"spectrum_number"],
        Mn_MCR_3[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.1, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Mn_MCR_3[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)


for i in range(Mn_MCR_3["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_St"]["pca_energy"],
        Mn_MCR_3["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCR_3["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCR_3["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCR_3["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCR_3["MCR_HM_residual"]["pca_energy"],
        Mn_MCR_3["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCR_3_2_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCR_3_2_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### 方案四，Hard-Model ("A -> B", "B + A -> C"), 利用 CLAESS Refers 确定初始浓度值，-20 到 80 eV, 0.633 和 0.367

In [ ]:
from tqdm.notebook import tqdm

reactions = ("A -> B", "A + B -> C")
species_concentrations = {"A": 0.633, "B": 0, "C": 0.367}
results = []


def evaluate_kinetics_2(k, m, XANES, ref_XANES):  # noqa: N803
    try:
        k0 = np.array([k, m])
        kin = scp.ActionMassKinetics(reactions, species_concentrations, k0)
        Ckin = kin.integrate(XANES.y.data)  # noqa: N806

        param_to_optimize = {"k[0]": k, "k[1]": m}
        mcr_4 = scp.MCRALS(
            log_level="ERROR",
            max_iter=30,
            maxdiv=5,
            tol=0.001,
            closureConc=[0, 1, 2],
            closureMethod="constantSum",
            closureTarget="default",  # closure constraints
            nonnegConc="all",
            solverConc="nnls",  # Non-negativity constraint
            nonnegSpec="all",
            solverSpec="nnls",  # Non-negativity constraint
        )
        mcr_4.hardConc = [0, 1, 2]
        mcr_4.getConc = kin.fit_to_concentrations
        mcr_4.argsGetConc = ([0, 1, 2], [0, 1, 2], param_to_optimize)
        mcr_4.kwargsGetConc = {
            "ivp_solver_kwargs": {"return_NDDataset": False, "method": "BDF"},
            "optimizer_kwargs": {"options": {"disp": False}},
        }
        mcr_4.fit(XANES, Ckin)
        std_st_Mn = np.std(np.asarray(mcr_4.St.data - ref_XANES[0].data), axis=1)  # noqa: N806
        return [k, m, std_st_Mn[0], std_st_Mn[1], std_st_Mn[2]], mcr_4
    except Exception as e:
        print(f"[Warning] k={k:.3f}, m={m:.3f} failed: {e}")
        return [
            None,
        ], None


for k in tqdm(np.linspace(0.0, 0.5, 100, endpoint=True)):
    for m in np.linspace(0.0, 0.5, 100, endpoint=True):
        result_row = evaluate_kinetics_2(k, m, XANES, ref_XANES)
        if result_row[0]:
            results.append(result_row[0])

result = pd.DataFrame(results, columns=["k", "m", "std_comp1", "std_comp2", "std_comp3"])

# 打印最小标准差对应的参数组合
for i in range(2, 5):
    print(result.iloc[result.iloc[:, i].idxmin(), :])  # type: ignore

# 保存结果
result.to_csv(
    Path.joinpath(path_out, r"P2b_Mn_MCRHM_2reaction_comp_3_A_B_B_AC.csv"),
    sep=",",
    index=False,
    header=True,
)

In [ ]:
# 将上述结果作为初始值，进行优化处理
# kinetic model
ka, kb = 0.075758, 0.000000  # 0.075758, 0.363636
reactions = ("A -> B", "B -> A + C")  # note the coma: this is now a tuple of size 1
species_concentrations = {"A": 0.69, "B": 0, "C": 0.31}

mcr_4 = evaluate_kinetics_2(ka, kb, XANES, ref_XANES)[1]  # 直接使用上面计算的结果
_ = mcr_4.C.T.plot(ylabel=r"relative ratio")
_ = mcr_4.C_constrained.T.plot(clear=False)
_ = mcr_4.St.plot(clear=True, ylims=(-0.1, 1.8))
_ = ref_XANES.plot(clear=False, ls="--", title=r"HM_MCR-ALS and refs")
_ = mcr_4.plotmerit(nb_traces=5)
mcr_4_residual = XANES - mcr_4.inverse_transform()

Mn_MCRHM_TworeactionData = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_4.C.data),
        "MCR_HM_C_constrained": (["spectrum_number", "mcr_number"], mcr_4.C_constrained.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_4.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_4.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_4_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_4.St.shape[0]),
    },
    attrs={
        "reactions": reactions,
        "species_concentrations": str(species_concentrations),
        "k0": str(ka) + ", " + str(kb),
    },
)

(
    xr.merge([Mn_NormalData, Mn_PCA, Mn_MCRHM_TworeactionData], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(
        Path.joinpath(path_out, r"P2b_Mn_MCRHM_2reaction_comp_3_A_B_B_AC.NETCDF4"), mode="w", engine="h5netcdf"
    )
)

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[3]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(1.0, 0.0, Mn_MCRHM_TworeactionData["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"#3"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Mn_MCRHM_TworeactionData[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Mn_MCRHM_TworeactionData[r"spectrum_number"],
        Mn_MCRHM_TworeactionData[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[0][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.1, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Mn_MCRHM_TworeactionData[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCRHM_TworeactionData["MCR_HM_St"].shape[0]):
    ax.plot(
        Mn_MCRHM_TworeactionData["MCR_HM_St"]["pca_energy"],
        Mn_MCRHM_TworeactionData["MCR_HM_St"][i, :],
        c=xascolors[0][i],
        label=labels[0][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Mn_NormalData["Reference"].shape[1]):
    ax.plot(
        Mn_NormalData["Reference"]["energy"],
        Mn_NormalData["Reference"][:, i],
        c=xascolors[0][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Mn_MCRHM_TworeactionData["MCR_HM_residual"].shape[0]):
    ax.plot(
        Mn_MCRHM_TworeactionData["MCR_HM_recon_Data"]["pca_energy"],
        Mn_MCRHM_TworeactionData["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Mn_MCRHM_TworeactionData["MCR_HM_residual"]["pca_energy"],
        Mn_MCRHM_TworeactionData["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(6530, 6630)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(-0.4, 1.6)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCRHM_TworeactionData_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2b_Mn_MCRHM_TworeactionData_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### Zn, 充电

In [ ]:
# Zn
data1 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Zn_Oct2022_1.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
data2 = pd.read_csv(  # noqa: N816
    Path.joinpath(path_file, r"Overview", r"cell2_opXAS_P2b_Zn_Oct2022_2.nor"),
    sep=r"\s+",
    index_col=None,
    header=None,
    comment="#",
)
xas_Zn = pd.concat([data1, data2.iloc[:, 3:]], axis=1, ignore_index=True)  # noqa: N816
charge_index = [0, 1, 2, *list(range(11, xas_Zn.shape[1], 1))]
xas_Zn = xas_Zn.iloc[:, charge_index]  # noqa: N816

pdf_Zn = xas_Zn.iloc[:, 0:3]  # noqa: N816
pdf_Zn.columns = [r"Energy", r"0.5M_ZnSO4(aq.)", r"ZHS"]
opData_Zn = xas_Zn.iloc[:, [0, *list(range(3, xas_Zn.shape[1]))]]  # noqa: N816
opData_Zn.columns = list(range(0, opData_Zn.shape[1], 1))

In [ ]:
Zn_NormalData = xr.Dataset(
    data_vars={
        "Spectrum": (["energy", "spectrum_number"], opData_Zn.iloc[:, 1:]),
        "Reference": (["energy", "reference_number"], pdf_Zn.iloc[:, 1:]),
    },
    coords={
        "spectrum_number": list(range(0, opData_Zn.shape[1] - 1, 1)),
        "energy": opData_Zn.iloc[:, 0],
        "reference_number": [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"],
    },
)

In [ ]:
data = scp.NDDataset(
    data=Zn_NormalData["Spectrum"].data.T,
    name="opXAS_Mn_CLAESS_2022",
    author="Cheng Liu, Ashely, and Dino Tonti",
    description="opXAS_Mn_CLAESS_2022",
    history="creation",
    title="absorption",
    units=ur.absorbance,
)

data.y = scp.Coord.arange(Zn_NormalData["Spectrum"].shape[1], title="sperctum number")
data.y.labels = Zn_NormalData.coords["spectrum_number"]
data.x = scp.Coord(Zn_NormalData.coords["energy"], title="energy", units=ur.eV)


ref = scp.NDDataset(data=Zn_NormalData["Reference"].data.T, name="ref_NOTOs_2024", units=ur.absorbance)
ref.y = scp.Coord.arange(Zn_NormalData["Reference"].shape[1], title="sperctum number")
ref.x = scp.Coord(Zn_NormalData.coords["energy"], title="energy", units=ur.eV)

_ = data[:, 9643.0:9743.0].plot(title="Normalized Data")
_ = ref[:, 9643.0:9743.0].plot(title=r"Normalized Data and Refs", ls="--", clear=False)

XANES = data[:, 9643.0:9743.0]
ref_XANES = ref[:, 9643.0:9743.0]  # noqa: N816


##### PCA

In [ ]:
# pca analysis
pca = scp.PCA()
pca.fit(XANES)
pca.printev()
# _ = pca.screeplot()  # noqa: ERA001
# loadings = pca.loadings  # noqa: ERA001
# scores = pca.scores  # noqa: ERA001
# _ = pca.loadings.plot(); _.set_ylabel('loadings')  # noqa: ERA001
# _ = pca.scores.T.plot(); _.set_ylabel('scores')  # noqa: ERA001
# _ =pca.scoreplot(pca.scores, 1, 2, color_mapping="labels")  # noqa: ERA001


In [ ]:
Zn_PCA = xr.Dataset(
    data_vars={
        "Explained_variance_ratio": (["spectrum_number"], pca.ev_ratio),
        "Cumulative_explained_variance": (["spectrum_number"], pca.ev_cum),
        "PCA_recon_Data": (["spectrum_number", "pca_energy"], pca.inverse_transform().data),
        "PCA_residual": (["spectrum_number", "pca_energy"], XANES.data - pca.inverse_transform().data),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
    },
)

##### Components = 2

##### 方案一，两个标样做约束

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]
    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr = scp.MCRALS(
    max_iter=100,
    maxdiv=30,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr.hardSpec = [0, 1]
mcr.getSpec = get_St
mcr.hardSt_to_St_idx = [0, 1]
mcr.fit(XANES, St0)
mcr.kwargsGetConc = {"ivp_solver_kwargs": {"return_NDDataset": False}, "optimizer_kwargs": {"options": {"disp": False}}}

_ = mcr.C.T.plot(clear=True, title=r"Concentration")
_ = mcr.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--")

_ = mcr.plotmerit(nb_traces=5, clear=True)
mcr_residual = XANES - mcr.inverse_transform()

In [ ]:
Zn_MCR_2 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr.St.shape[0]),
    },
)

(xr.merge([Zn_NormalData, Zn_PCA, Zn_MCR_2], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2b_Zn_MCR_2.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4]], [colors[2], colors[1]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(0.0, 1.0, Zn_MCR_2["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Zn_MCR_2[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Zn_MCR_2[r"spectrum_number"],
        Zn_MCR_2[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[1][i],
        markeredgecolor=xascolors[1][i],
        markersize=8,
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.5, 0.6),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Zn_MCR_2[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_NormalData["Reference"].shape[1]):
    ax.plot(
        Zn_NormalData["Reference"]["energy"],
        Zn_NormalData["Reference"][:, i],
        c=xascolors[1][i],
        ls="--",
        lw=1.0,
        label=labels[1][i],
        zorder=5,
    )
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(Zn_MCR_2["MCR_HM_St"].shape[0]):
    ax.plot(
        Zn_MCR_2["MCR_HM_St"]["pca_energy"],
        Zn_MCR_2["MCR_HM_St"][i, :],
        c=xascolors[1][i],
        label=None,
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_MCR_2["MCR_HM_residual"].shape[0]):
    ax.plot(
        Zn_MCR_2["MCR_HM_recon_Data"]["pca_energy"],
        Zn_MCR_2["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Zn_MCR_2["MCR_HM_residual"]["pca_energy"],
        Zn_MCR_2["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(-0.4, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2b_Zn_MCR_2_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2b_Zn_MCR_2_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### 方案二， Hard-Model ( A --> B ), ZHS 0.688, 和 0.5M Zn2+ 0.312

In [ ]:
# kinetic model
from tqdm.notebook import tqdm

reactions = ("A -> B",)  # note the coma: this is now a tuple of size 1, 不管方程式的系数：这个从 Zn 那边验证过
species_concentrations = {"A": 0.688, "B": 0.312}
results = []


def evaluate_kinetics(k, XANES, ref_XANES):  # noqa: N803
    try:
        k0 = np.array([
            k,
        ])  # as we have a single reaction, we must use a single rate constant.
        kin = scp.ActionMassKinetics(reactions, species_concentrations, k0)
        Ckin = kin.integrate(XANES.y.data)  # noqa: N806

        param_to_optimize = {
            "k[0]": k,
        }
        mcr_2 = scp.MCRALS(
            max_iter=100,
            maxdiv=30,
            log_level="ERROR",
            tol=0.001,
            closureConc=[0, 1],
            closureMethod="constantSum",
            closureTarget="default",  # closure constraints
            nonnegConc="all",
            solverConc="nnls",  # Non-negativity constraint
            nonnegSpec="all",
            solverSpec="nnls",  # Non-negativity constraint
        )
        mcr_2.hardConc = [0, 1]
        mcr_2.getConc = kin.fit_to_concentrations
        mcr_2.argsGetConc = ([0, 1], [0, 1], param_to_optimize)
        mcr_2.kwargsGetConc = {
            "ivp_solver_kwargs": {"return_NDDataset": False},
            "optimizer_kwargs": {"options": {"disp": False}},
        }
        mcr_2.fit(XANES, Ckin)
        std_st_Zn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1)  # Zn2+  # noqa: N806
        return [k, std_st_Zn[0], std_st_Zn[1]], mcr_2
        # std_st_Zn = np.std(np.asarray(mcr_2.St.data - ref_XANES[0].data), axis=1) # Zn2+ 作为参考  # noqa: ERA001
        # std_st_Zn2 = np.std(np.asarray(mcr_2.St.data - ref_XANES[1].data), axis=1) # ZHS 作为参考  # noqa: ERA001
        # std_Mn= [std_st_Zn[i] + std_st_Zn2[j] for i in range(len(std_st_Zn)) for j in range(len(std_st_Zn2)) if i != j]  # noqa: E501, ERA001
        # return [k, std_st_Zn[0], std_st_Zn[1]], mcr_2  # noqa: ERA001
    except Exception as e:
        print(f"[Warning] k={k:.3f}, failed: {e}")
        return [
            None,
        ], None


for k in tqdm(np.linspace(0.0, 0.5, 500, endpoint=True)):
    result_row = evaluate_kinetics(k, XANES, ref_XANES)
    if result_row[0]:
        results.append(result_row[0])

result = pd.DataFrame(results, columns=["k", "std_comp1", "std_comp2"])

# 打印最小标准差对应的参数组合
for i in range(1, 3):
    print(result.iloc[result.iloc[:, i].idxmin(), :])  # type: ignore

# 保存结果
result.to_csv(
    Path.joinpath(path_out, r"P2b_Zn_MCRHM_1reaction_comp_2_A_B.csv"),
    sep=",",
    index=False,
    header=True,
)

In [ ]:
# 将上述结果作为初始值，进行最后的结果
#  kinetic model
ka = 0.126253
reactions = ("A -> B",)  # note the coma: this is now a tuple of size 1
species_concentrations = {"A": 0.688, "B": 0.312}

mcr_2 = evaluate_kinetics(ka, XANES, ref_XANES)[1]  # 直接使用上面计算的结果
_ = mcr_2.C.T.plot(ylabel=r"relative ratio")
_ = mcr_2.C_constrained.T.plot(clear=False)
_ = mcr_2.St.plot(clear=True, ylims=(-0.1, 1.8))
_ = ref_XANES.plot(clear=False, ls="--")

_ = mcr_2.plotmerit(nb_traces=5)
mcr_2_residual = XANES - mcr_2.inverse_transform()

Zn_MCRHM_OnereactionData = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_2.C.data),
        "MCR_HM_C_constrained": (["spectrum_number", "mcr_number"], mcr_2.C_constrained.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_2.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_2.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_2_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_2.St.shape[0]),
    },
    attrs={
        "reactions": reactions,
        "species_concentrations": str(species_concentrations),
        "k0": ka,
    },
)

(
    xr.merge([Zn_NormalData, Zn_PCA, Zn_MCRHM_OnereactionData], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(
        Path.joinpath(path_out, r"P2b_Zn_MCRHM_1reaction_comp_2_A_B_2.NETCDF4"), mode="w", engine="h5netcdf"
    )
)

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 3.3))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4]], [colors[1], colors[2]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(0.0, 1.0, Zn_MCRHM_OnereactionData["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [[r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$"], [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS"]]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Zn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Zn_MCRHM_OnereactionData[r"spectrum_number"],
        Zn_MCRHM_OnereactionData[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[1][i],
        markeredgecolor=xascolors[1][i],
        markersize=8,
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.3, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

line = []
for i in range(Zn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"].shape[1]):
    (lineB,) = ax.plot(  # noqa: N816
        Zn_MCRHM_OnereactionData[r"spectrum_number"],
        Zn_MCRHM_OnereactionData[r"MCR_HM_C_constrained"][:, i],
        c="b",
        label=labels[1][i],
        ls="--",
    )
    line.append(lineB)
ax.legend(
    handles=[line[0]],
    labels=[r"Initial Concentration"],
    loc="upper left",
    bbox_to_anchor=(0.1, 0.15),
    ncols=1,
    handlelength=3,
    labelcolor="linecolor",
    fontsize=9,
)

ax.set_xlim(-0.5, Zn_MCRHM_OnereactionData[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Zn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_NormalData["Reference"].shape[1]):
    ax.plot(
        Zn_NormalData["Reference"]["energy"],
        Zn_NormalData["Reference"][:, i],
        c=xascolors[1][i],
        ls="--",
        lw=1.0,
        label=labels[1][i],
        zorder=5,
    )
ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
for i in range(Zn_MCRHM_OnereactionData["MCR_HM_St"].shape[0]):
    ax.plot(
        Zn_MCRHM_OnereactionData["MCR_HM_St"]["pca_energy"],
        Zn_MCRHM_OnereactionData["MCR_HM_St"][i, :],
        c=xascolors[1][i],
        label=None,
        ls="-",
        lw=1.0,
    )

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.6, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_MCRHM_OnereactionData["MCR_HM_residual"].shape[0]):
    ax.plot(
        Zn_MCRHM_OnereactionData["MCR_HM_recon_Data"]["pca_energy"],
        Zn_MCRHM_OnereactionData["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Zn_MCRHM_OnereactionData["MCR_HM_residual"]["pca_energy"],
        Zn_MCRHM_OnereactionData["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(-0.4, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2b_Zn_MCRHM_1reaction_comp_2_A_B_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2b_Zn_MCRHM_1reaction_comp_2_A_B_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

##### Components = 3

##### 方案一，两个标样做约束

In [ ]:
def get_St(St):  # St must be passed, even if not actuially used  # noqa: N802, N803
    St[0, :] = ref_XANES[0]
    St[1, :] = ref_XANES[1]
    return St


St0 = np.stack(
    (
        ref_XANES[0].squeeze().data,
        ref_XANES[1].squeeze().data,
        XANES[-1].squeeze().data,
    ),
    axis=0,
)

St0 = scp.NDDataset(
    St0,
    name="Dataset initial guesses spectrum",
    author="ChengLiu",
    description="Initial spectrum guesses",
)

mcr_3 = scp.MCRALS(
    max_iter=300,
    maxdiv=100,
    log_level="ERROR",
    tol=0.001,
    closureConc=[0, 1, 2],
    closureMethod="constantSum",
    closureTarget="default",  # closure constraints
    nonnegConc="all",
    solverConc="nnls",  # Non-negativity constraint
    nonnegSpec="all",
    solverSpec="nnls",  # Non-negativity constraint
)
mcr_3.hardSpec = [0, 1, 2]
mcr_3.getSpec = get_St
mcr_3.hardSt_to_St_idx = [0, 1, 2]
mcr_3.fit(XANES, St0)
mcr_3.kwargsGetConc = {
    "ivp_solver_kwargs": {"return_NDDataset": False},
    "optimizer_kwargs": {"options": {"disp": False}},
}

_ = mcr_3.C.T.plot(clear=True, title=r"Concentration")
_ = mcr_3.St.plot(clear=True)
_ = ref_XANES.plot(clear=False, ls="--")

_ = mcr_3.plotmerit(nb_traces=5, clear=True)
mcr_3_residual = XANES - mcr_3.inverse_transform()

Zn_MCR_3 = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_3.C.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_3.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_3.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_3_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_3.St.shape[0]),
    },
)

(xr.merge([Zn_NormalData, Zn_PCA, Zn_MCR_3], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(Path.joinpath(path_out, r"P2b_Zn_MCR_3.NETCDF4"), mode="w", engine="h5netcdf"))

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[3]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(0.0, 1.0, Zn_MCR_3["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"#3"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Zn_MCR_3[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Zn_MCR_3[r"spectrum_number"],
        Zn_MCR_3[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[1][i],
        markeredgecolor=xascolors[1][i],
        markersize=8,
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.0, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Zn_MCR_3[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=5, offset=0))
ax.set_ylim(-0.05, 1.1)
ax.set_ylabel(r"Zn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=-0.1))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=-0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)


for i in range(Zn_MCR_3["MCR_HM_St"].shape[0]):
    ax.plot(
        Zn_MCR_3["MCR_HM_St"]["pca_energy"],
        Zn_MCR_3["MCR_HM_St"][i, :],
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Zn_NormalData["Reference"].shape[1]):
    ax.plot(
        Zn_NormalData["Reference"]["energy"],
        Zn_NormalData["Reference"][:, i],
        c=xascolors[1][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_MCR_3["MCR_HM_residual"].shape[0]):
    ax.plot(
        Zn_MCR_3["MCR_HM_recon_Data"]["pca_energy"],
        Zn_MCR_3["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Zn_MCR_3["MCR_HM_residual"]["pca_energy"],
        Zn_MCR_3["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(-0.4, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2b_Zn_MCR_3_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2b_Zn_MCR_3_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()


##### 方案二，Hard-Model ( "A -> B", "B -> C" ), 利用 CLAESS Refers 确定初始浓度值，-20 到 80 eV, 1.0 和 0.0

In [ ]:
from tqdm.notebook import tqdm

reactions = ("A -> B", "B -> C")
species_concentrations = {"A": 0.688, "B": 0.312, "C": 0.0}
results = []


def evaluate_kinetics_2(k, m, XANES, ref_XANES):  # noqa: N803
    try:
        k0 = np.array([k, m])
        kin = scp.ActionMassKinetics(reactions, species_concentrations, k0)
        Ckin = kin.integrate(XANES.y.data)  # noqa: N806

        param_to_optimize = {"k[0]": k, "k[1]": m}
        mcr_4 = scp.MCRALS(
            log_level="ERROR",
            max_iter=30,
            maxdiv=5,
            tol=0.001,
            closureConc=[0, 1, 2],
            closureMethod="constantSum",
            closureTarget="default",  # closure constraints
            nonnegConc="all",
            solverConc="nnls",  # Non-negativity constraint
            nonnegSpec="all",
            solverSpec="nnls",  # Non-negativity constraint
        )
        mcr_4.hardConc = [0, 1, 2]
        mcr_4.getConc = kin.fit_to_concentrations
        mcr_4.argsGetConc = ([0, 1, 2], [0, 1, 2], param_to_optimize)
        mcr_4.kwargsGetConc = {
            "ivp_solver_kwargs": {"return_NDDataset": False, "method": "BDF"},
            "optimizer_kwargs": {"options": {"disp": False}},
        }
        mcr_4.fit(XANES, Ckin)
        std_st_Mn = np.std(np.asarray(mcr_4.St.data - ref_XANES[0].data), axis=1)  # noqa: N806
        return [k, m, std_st_Mn[0], std_st_Mn[1], std_st_Mn[2]], mcr_4
    except Exception as e:
        print(f"[Warning] k={k:.3f}, m={m:.3f} failed: {e}")
        return [
            None,
        ], None


for k in tqdm(np.linspace(0.0, 0.5, 100, endpoint=True)):
    for m in np.linspace(0.0, 0.5, 100, endpoint=True):
        result_row = evaluate_kinetics_2(k, m, XANES, ref_XANES)
        if result_row[0]:
            results.append(result_row[0])

result = pd.DataFrame(results, columns=["k", "m", "std_comp1", "std_comp2", "std_comp3"])

# 打印最小标准差对应的参数组合
for i in range(2, 5):
    print(result.iloc[result.iloc[:, i].idxmin(), :])  # type: ignore

# 保存结果
result.to_csv(
    Path.joinpath(path_out, r"P2b_Zn_MCRHM_2reaction_comp_3_A_B_B_C.csv"),
    sep=",",
    index=False,
    header=True,
)

In [ ]:
# 将上述结果作为初始值，进行优化处理
# kinetic model
ka, kb = 0.459596, 0.212121
reactions = ("A -> B", "B -> C")
species_concentrations = {"A": 1.0, "B": 0.0, "C": 0.0}

mcr_4 = evaluate_kinetics_2(ka, kb, XANES, ref_XANES)[1]  # 直接使用上面计算的结果
_ = mcr_4.C.T.plot(ylabel=r"relative ratio")
_ = mcr_4.C_constrained.T.plot(clear=False)
_ = mcr_4.St.plot(clear=True, ylims=(-0.1, 1.8))
_ = ref_XANES.plot(clear=False, ls="--")
_ = mcr_4.plotmerit(nb_traces=5)
mcr_4_residual = XANES - mcr_4.inverse_transform()

Zn_MCRHM_TworeactionData = xr.Dataset(
    data_vars={
        "MCR_HM_C": (["spectrum_number", "mcr_number"], mcr_4.C.data),
        "MCR_HM_C_constrained": (["spectrum_number", "mcr_number"], mcr_4.C_constrained.data),
        "MCR_HM_St": (["mcr_number", "pca_energy"], mcr_4.St.data),
        "MCR_HM_recon_Data": (["spectrum_number", "pca_energy"], mcr_4.inverse_transform().data),
        "MCR_HM_residual": ((["spectrum_number", "pca_energy"], mcr_4_residual.data)),
    },
    coords={
        "spectrum_number": XANES.y.data,
        "pca_energy": XANES.x.data,
        "mcr_number": np.arange(mcr_4.St.shape[0]),
    },
    attrs={
        "reactions": reactions,
        "species_concentrations": str(species_concentrations),
        "k0": str(ka) + ", " + str(kb),
    },
)

(
    xr.merge([Zn_NormalData, Zn_PCA, Zn_MCRHM_TworeactionData], compat="no_conflicts", combine_attrs="no_conflicts").to_netcdf(
        Path.joinpath(path_out, r"P2b_Zn_MCRHM_2reaction_comp_3_A_B_B_C.NETCDF4"), mode="w", engine="h5netcdf"
    )
)

In [ ]:
# 画图
# gridspec inside gridspec
plt.close("all")
%matplotlib inline
fig = plt.figure(figsize=(7, 2.5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

xascolors = [[colors[0], colors[4], colors[3]], [colors[2], colors[1], colors[5]]]
xas_colors = ListedColormap(
    mpl.colormaps["coolwarm"](np.linspace(0.0, 1.0, Zn_MCRHM_TworeactionData["MCR_HM_residual"].shape[0])),
    name="xas_colors",
)
labels = [
    [r"0.2M Mn$\mathrm{^{2\!+}}$", r"${\alpha}$-MnO$\mathrm{_2}$", r"#3"],
    [r"0.5M Zn$\mathrm{^{2\!+}}$", r"ZHS", r"#3"],
]

# 图 A
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.0, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

line = []
for i in range(Zn_MCRHM_TworeactionData[r"MCR_HM_C"].shape[1]):
    (lineA,) = ax.plot(  # noqa: N816
        Zn_MCRHM_TworeactionData[r"spectrum_number"],
        Zn_MCRHM_TworeactionData[r"MCR_HM_C"][:, i],
        marker="o",
        markerfacecolor=xascolors[1][i],
        markeredgecolor=xascolors[0][i],
        markersize=8,
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
    )
    line.append(lineA)
first_legend = ax.legend(
    handles=line,
    loc="upper left",
    bbox_to_anchor=(0.1, 1.0),
    ncols=1,
    frameon=False,
    labelcolor="linecolor",
    fontsize=9,
)
# Add the legend manually to the Axes.
ax.add_artist(first_legend)

ax.set_xlim(-0.5, Zn_MCRHM_TworeactionData[r"MCR_HM_C"].shape[0])
ax.set_xlabel(r"Spectrum Number", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=1, offset=0))
ax.set_ylim(0.0, 1.0)
ax.set_ylabel(r"Mn Molar Fraction (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1))
ax.tick_params(
    axis="both",
    which="both",
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labelleft=True,
    labeltop=False,
    labelright=False,
)

# 图 B
subfig = fig.add_subfigure(gs[0, 1], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.3, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_MCRHM_TworeactionData["MCR_HM_St"].shape[0]):
    ax.plot(
        Zn_MCRHM_TworeactionData["MCR_HM_St"]["pca_energy"],
        Zn_MCRHM_TworeactionData["MCR_HM_St"][i, :],
        c=xascolors[1][i],
        label=labels[1][i],
        ls="-",
        lw=1.0,
        alpha=0.5,
    )

for i in range(Zn_NormalData["Reference"].shape[1]):
    ax.plot(
        Zn_NormalData["Reference"]["energy"],
        Zn_NormalData["Reference"][:, i],
        c=xascolors[1][i],
        ls="--",
        lw=1.0,
        label=None,
        zorder=5,
    )
ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=10))
ax.set_ylim(0, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)
ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

# 图 C
subfig = fig.add_subfigure(gs[0, 2], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0.6, 0.0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(Zn_MCRHM_TworeactionData["MCR_HM_residual"].shape[0]):
    ax.plot(
        Zn_MCRHM_TworeactionData["MCR_HM_recon_Data"]["pca_energy"],
        Zn_MCRHM_TworeactionData["MCR_HM_recon_Data"][i, :],
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )
    ax.plot(
        Zn_MCRHM_TworeactionData["MCR_HM_residual"]["pca_energy"],
        Zn_MCRHM_TworeactionData["MCR_HM_residual"][i, :] - 0.2,
        c=xas_colors.colors[i],
        label=None,
        ls="-",
        lw=1.0,  # noqa: E501 # type: ignore
    )

# Add colorbar and adjust ticks afterwards
colorbar = Colorbar(
    ax=ax.inset_axes((0.5, 0.15, 0.42, 0.05)),
    location="bottom",
    orientation="horizontal",
    cmap=xas_colors,
    ticklocation="top",
    spacing="proportional",
    drawedges=False,
)
colorbar.set_ticks([])

ax.text(
    0.49,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="left",
    fontfamily="Arial",
)
ax.text(
    0.84,
    0.36,
    r"Discharge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
ax.text(
    0.92,
    0.28,
    r"Charge",
    transform=ax.transAxes,
    fontsize=9,
    va="top",
    ha="right",
    fontfamily="Arial",
)
colorbar.outline.set_visible(False)  # type: ignore

ax.set_xlim(9645, 9745)
ax.set_xlabel(r"Energy (eV)", fontsize=11)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=5))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=5))
ax.set_ylim(-0.4, 2.4)
ax.set_ylabel(r"Absorption (arb.u.)", fontsize=11)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.4))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.2))

ax.legend(loc="upper left", bbox_to_anchor=(0.5, 1.0), ncols=1, handlelength=3, labelcolor="linecolor", fontsize=9)

ax.tick_params(
    axis="both",
    which="both",
    labelsize=9,
    direction="out",
    bottom=True,
    top=False,
    left=True,
    right=False,
    labelbottom=True,
    labeltop=False,
    labelleft=True,
    labelright=False,
)

plt.savefig(
    Path.joinpath(path_out, r"P2b_Zn_MCRHM_TworeactionData_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"P2b_Zn_MCRHM_TworeactionData_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()